## Introductory Steps

In [1]:
!pip install arch

In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import yfinance as yf
import arch
from arch import arch_model
from scipy.stats import norm

In [3]:
from google.colab import drive
drive.mount('/content/drive')
import os

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
os.chdir('/content/drive/My Drive/thesis/final_submission')
df = pd.read_csv('non_nan_4.csv')

In [5]:
df

,Date,unemployment_rate,CPI,treasury_yield,GDP_growth,SP500_return,AZN,BMY,JNJ,LLY,MRK,NVO,NVS,PFE,ROG,inflation_change,unemp_change,treasury_yield_change
0,2000-02-01,4.1,170.000,6.661000,0.496560,-1.522563,-12.828964,-13.228004,-16.339821,-11.121498,-21.701151,2.220031,3.838386,-11.226228,54.440789,1.000,-0.1,-0.141500
1,2000-03-01,4.0,171.000,6.519500,0.511258,9.413333,22.264136,-0.218329,-2.079067,5.804243,0.913712,8.390897,6.420237,14.101954,6.922258,1.000,-0.1,-0.141500
2,2000-04-01,3.8,170.900,6.256522,1.327803,-3.266805,5.567379,-8.205683,17.437698,23.153694,12.400712,-0.097663,2.559423,15.213674,7.370518,-0.100,-0.2,-0.262978
3,2000-05-01,4.0,171.200,5.990526,-0.181797,-1.572223,-0.148357,5.395746,8.484832,-1.296597,7.374072,20.863985,5.169310,5.638019,-8.163265,0.300,0.2,-0.265995
4,2000-06-01,4.0,172.200,6.440455,0.305565,1.728613,10.549735,5.788826,14.239888,31.641749,3.078671,2.813690,8.474599,8.076012,13.131313,1.000,0.0,0.449928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,2023-01-01,3.4,300.536,3.616190,0.390254,6.776820,-3.584079,0.972908,-7.489384,-5.928822,-2.549213,2.541749,-0.110227,-13.817335,16.968326,1.546,-0.1,-0.274810
276,2023-02-01,3.6,301.648,3.531500,-0.442183,-2.514271,-0.290649,-4.328217,-6.217115,-9.568502,-1.089288,1.592445,-7.172811,-7.286115,5.451681,1.112,0.2,-0.084690
277,2023-03-01,3.5,301.808,3.746842,-0.442183,3.313488,8.035329,0.507544,1.862736,10.703390,0.141189,12.873250,9.367574,0.566924,11.025813,0.160,-0.1,0.215342
278,2023-04-01,3.4,302.918,3.663043,-0.442183,1.985238,5.489119,-3.664707,5.612908,15.269915,9.289214,5.836894,16.334413,-4.681371,-1.517467,1.110,-0.1,-0.083799


In [6]:
def create_lags_cov(data, covs, period):
    lagged_covs = []  # List to store the names of lagged columns

    for covariate in covs:
        for i in period:
            lagged_column_name = f'{covariate}_lag{i}'
            data[lagged_column_name] = data[covariate].shift(i)
            lagged_covs.append(lagged_column_name)

    data.dropna(inplace=True)
    data = data.reset_index(drop=True)

    return data, lagged_covs

In [7]:
def create_lags_returns(data, tickers, period):

    lagged_tickers = []  # List to store the names of lagged columns

    for tick in tickers:
        for i in period:
            lagged_column_name = f'{tick}_lag{i}'
            data[lagged_column_name] = data[tick].shift(i)
            lagged_tickers.append(lagged_column_name)

    data.dropna(inplace=True)
    data = data.reset_index(drop=True)

    return data, lagged_tickers

In [8]:
# Creating lagged variables
tickers = ['AZN', 'BMY', 'JNJ', 'LLY', 'MRK', 'NVO', 'NVS', 'PFE','ROG']
covs = ['unemployment_rate', 'CPI', 'treasury_yield', 'GDP_growth', 'SP500_return', 'inflation_change', 'unemp_change', 'treasury_yield_change']
period_covs = [1, 2, 3]
period_returns = [1,2]

data, lagged_covs = create_lags_cov(df, covs, period=period_covs)
data, lagged_tickers = create_lags_returns(df, tickers, period= period_returns)

In [9]:
# Creating dummy variables
data['Quater_dummy'] = 0
data['Quater_rippel'] = 0
data['Date'] = pd.to_datetime(data['Date'])
for index, row in data.iterrows():
    if row['Date'].month in [3, 6, 9, 12]:
        data.loc[index, 'Quater_dummy'] = 1
    if row['Date'].month in [1, 4, 7, 10]:
        data.loc[index, 'Quater_dummy'] = 1

In [10]:
data

,Date,unemployment_rate,CPI,treasury_yield,GDP_growth,SP500_return,AZN,BMY,JNJ,LLY,...,NVO_lag1,NVO_lag2,NVS_lag1,NVS_lag2,PFE_lag1,PFE_lag2,ROG_lag1,ROG_lag2,Quater_dummy,Quater_rippel
0,2000-07-01,4.0,172.700,6.097273,-0.653855,-1.338453,-8.064479,-11.373392,-8.650271,4.005041,...,2.813690,20.863985,8.474599,5.169310,8.076012,5.638019,13.131313,-8.163265,1,0
1,2000-08-01,4.1,172.700,6.054000,0.795710,6.534124,6.578935,3.131245,-1.192113,-29.708189,...,13.757382,2.813690,-3.593737,8.474599,-9.635421,8.076012,2.142857,13.131313,0,0
2,2000-09-01,3.9,173.600,5.826087,0.249670,-5.723123,15.363525,8.458631,2.496165,11.482185,...,5.591665,13.757382,-1.944901,-3.593737,-0.540360,-9.635421,-2.447552,2.142857,1,0
3,2000-10-01,3.9,173.900,5.799000,0.187218,-0.217758,-8.764862,5.978226,-1.929509,10.169493,...,9.359631,5.591665,2.314016,-1.944901,4.352595,-0.540360,-9.318996,-2.447552,1,0
4,2000-11-01,3.9,174.200,5.738571,0.009919,-7.465269,7.863717,14.237843,8.548194,4.825204,...,-4.279274,9.359631,-0.807726,2.314016,-3.860903,4.352595,13.043478,-9.318996,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,2023-01-01,3.4,300.536,3.616190,0.390254,6.776820,-3.584079,0.972908,-7.489384,-5.928822,...,8.619586,14.479974,1.295221,10.390734,3.090302,7.690670,9.446071,-53.665066,1,0
271,2023-02-01,3.6,301.648,3.531500,-0.442183,-2.514271,-0.290649,-4.328217,-6.217115,-9.568502,...,2.541749,8.619586,-0.110227,1.295221,-13.817335,3.090302,16.968326,9.446071,0,0
272,2023-03-01,3.5,301.808,3.746842,-0.442183,3.313488,8.035329,0.507544,1.862736,10.703390,...,1.592445,2.541749,-7.172811,-0.110227,-7.286115,-13.817335,5.451681,16.968326,1,0
273,2023-04-01,3.4,302.918,3.663043,-0.442183,1.985238,5.489119,-3.664707,5.612908,15.269915,...,12.873250,1.592445,9.367574,-7.172811,0.566924,-7.286115,11.025813,5.451681,1,0


## Defining loss function

In [11]:
def tick_loss(alpha, returns, var):
    df = pd.DataFrame({'Return': returns, 'VaR': var})
    df['Indicator'] = np.where(df['Return'] < df['VaR'], 1, 0)

    t_loss = 0

    for i in df.index:
        t_loss += (
            alpha * (df['Return'][i] - df['VaR'][i]) * (1 - df['Indicator'][i])
            + (1 - alpha) * (df['VaR'][i] - df['Return'][i]) * df['Indicator'][i]
        )

    return t_loss

In [12]:
data.columns

Index(['Date', 'unemployment_rate', 'CPI', 'treasury_yield', 'GDP_growth',
       'SP500_return', 'AZN', 'BMY', 'JNJ', 'LLY', 'MRK', 'NVO', 'NVS', 'PFE',
       'ROG', 'inflation_change', 'unemp_change', 'treasury_yield_change',
       'unemployment_rate_lag1', 'unemployment_rate_lag2',
       'unemployment_rate_lag3', 'CPI_lag1', 'CPI_lag2', 'CPI_lag3',
       'treasury_yield_lag1', 'treasury_yield_lag2', 'treasury_yield_lag3',
       'GDP_growth_lag1', 'GDP_growth_lag2', 'GDP_growth_lag3',
       'SP500_return_lag1', 'SP500_return_lag2', 'SP500_return_lag3',
       'inflation_change_lag1', 'inflation_change_lag2',
       'inflation_change_lag3', 'unemp_change_lag1', 'unemp_change_lag2',
       'unemp_change_lag3', 'treasury_yield_change_lag1',
       'treasury_yield_change_lag2', 'treasury_yield_change_lag3', 'AZN_lag1',
       'AZN_lag2', 'BMY_lag1', 'BMY_lag2', 'JNJ_lag1', 'JNJ_lag2', 'LLY_lag1',
       'LLY_lag2', 'MRK_lag1', 'MRK_lag2', 'NVO_lag1', 'NVO_lag2', 'NVS_lag1',
       

# Naïve Model Specifications

## 1. Macros Only

In [13]:
data_macro = data.copy()
to_exclude = []

for t in tickers:
    to_exclude.append(t)
    to_exclude.append(f'{t}_lag1')
    to_exclude.append(f'{t}_lag2')

for covariate in covs:
    to_exclude.append(covariate)

to_exclude.append('Date')

In [14]:
data_macro

,Date,unemployment_rate,CPI,treasury_yield,GDP_growth,SP500_return,AZN,BMY,JNJ,LLY,...,NVO_lag1,NVO_lag2,NVS_lag1,NVS_lag2,PFE_lag1,PFE_lag2,ROG_lag1,ROG_lag2,Quater_dummy,Quater_rippel
0,2000-07-01,4.0,172.700,6.097273,-0.653855,-1.338453,-8.064479,-11.373392,-8.650271,4.005041,...,2.813690,20.863985,8.474599,5.169310,8.076012,5.638019,13.131313,-8.163265,1,0
1,2000-08-01,4.1,172.700,6.054000,0.795710,6.534124,6.578935,3.131245,-1.192113,-29.708189,...,13.757382,2.813690,-3.593737,8.474599,-9.635421,8.076012,2.142857,13.131313,0,0
2,2000-09-01,3.9,173.600,5.826087,0.249670,-5.723123,15.363525,8.458631,2.496165,11.482185,...,5.591665,13.757382,-1.944901,-3.593737,-0.540360,-9.635421,-2.447552,2.142857,1,0
3,2000-10-01,3.9,173.900,5.799000,0.187218,-0.217758,-8.764862,5.978226,-1.929509,10.169493,...,9.359631,5.591665,2.314016,-1.944901,4.352595,-0.540360,-9.318996,-2.447552,1,0
4,2000-11-01,3.9,174.200,5.738571,0.009919,-7.465269,7.863717,14.237843,8.548194,4.825204,...,-4.279274,9.359631,-0.807726,2.314016,-3.860903,4.352595,13.043478,-9.318996,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,2023-01-01,3.4,300.536,3.616190,0.390254,6.776820,-3.584079,0.972908,-7.489384,-5.928822,...,8.619586,14.479974,1.295221,10.390734,3.090302,7.690670,9.446071,-53.665066,1,0
271,2023-02-01,3.6,301.648,3.531500,-0.442183,-2.514271,-0.290649,-4.328217,-6.217115,-9.568502,...,2.541749,8.619586,-0.110227,1.295221,-13.817335,3.090302,16.968326,9.446071,0,0
272,2023-03-01,3.5,301.808,3.746842,-0.442183,3.313488,8.035329,0.507544,1.862736,10.703390,...,1.592445,2.541749,-7.172811,-0.110227,-7.286115,-13.817335,5.451681,16.968326,1,0
273,2023-04-01,3.4,302.918,3.663043,-0.442183,1.985238,5.489119,-3.664707,5.612908,15.269915,...,12.873250,1.592445,9.367574,-7.172811,0.566924,-7.286115,11.025813,5.451681,1,0


In [15]:
to_exclude

['AZN',
 'AZN_lag1',
 'AZN_lag2',
 'BMY',
 'BMY_lag1',
 'BMY_lag2',
 'JNJ',
 'JNJ_lag1',
 'JNJ_lag2',
 'LLY',
 'LLY_lag1',
 'LLY_lag2',
 'MRK',
 'MRK_lag1',
 'MRK_lag2',
 'NVO',
 'NVO_lag1',
 'NVO_lag2',
 'NVS',
 'NVS_lag1',
 'NVS_lag2',
 'PFE',
 'PFE_lag1',
 'PFE_lag2',
 'ROG',
 'ROG_lag1',
 'ROG_lag2',
 'unemployment_rate',
 'CPI',
 'treasury_yield',
 'GDP_growth',
 'SP500_return',
 'inflation_change',
 'unemp_change',
 'treasury_yield_change',
 'Date']

### 1.1. 1-step-ahead VaR: In-sample forecast for 1 month

In [14]:
def rolling_1m_forecast(data, dates, ticker, alpha, columns_to_exclude):
    t_loss = 0
    dta = data.copy()

    for d in dates:
        train_df = dta[dta.Date <= d]
        test_df = dta[dta.Date > d]
        test_df = test_df.reset_index(drop=True)

        X_train = train_df.drop(columns=columns_to_exclude)
        y_train = train_df[ticker]

        X_test = test_df.drop(columns=columns_to_exclude)
        X_test = X_test.iloc[0]
        y_test = test_df[ticker]

        garch_model = arch_model(y_train, vol='Garch', p=1, q=1)
        garch_results = garch_model.fit()

        forecasts = garch_results.forecast(start=0, horizon=1)
        cond_vol_forecast = np.sqrt(forecasts.variance.iloc[-1])
        mean_forecast = forecasts.mean.iloc[-1]

        VaR_forecast = mean_forecast + cond_vol_forecast * norm.ppf(alpha)

        loss = tick_loss(alpha, y_test.iloc[0], VaR_forecast)

        t_loss += loss

    t_loss = t_loss / len(dates)

    return t_loss

In [15]:
dates1m = ['2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01', '2022-09-01', '2022-10-01', '2022-11-01', '2022-12-01', '2023-01-01', '2023-02-01', '2023-03-01', '2023-04-01']

In [23]:
for t in tickers:
    loss = rolling_1m_forecast(data, dates1m, t, alpha=0.05, columns_to_exclude= to_exclude)
    print(f'{t}-1m-loss: {loss} \n')

Iteration:      1,   Func. Count:      6,   Neg. LLF: 874.1785677557702
Iteration:      2,   Func. Count:     12,   Neg. LLF: 1200.4721531726964
Iteration:      3,   Func. Count:     19,   Neg. LLF: 870.6852289593519
Iteration:      4,   Func. Count:     26,   Neg. LLF: 867.8399619926613
Iteration:      5,   Func. Count:     32,   Neg. LLF: 867.828481407204
Iteration:      6,   Func. Count:     37,   Neg. LLF: 867.8058039900275
Iteration:      7,   Func. Count:     42,   Neg. LLF: 867.748173849943
Iteration:      8,   Func. Count:     47,   Neg. LLF: 867.7462745676646
Iteration:      9,   Func. Count:     52,   Neg. LLF: 867.7461706287772
Iteration:     10,   Func. Count:     57,   Neg. LLF: 867.746169625292
Iteration:     11,   Func. Count:     61,   Neg. LLF: 867.7461696251956
Optimization terminated successfully    (Exit mode 0)
            Current function value: 867.746169625292
            Iterations: 11
            Function evaluations: 61
            Gradient evaluations: 11
It

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:     10,   Func. Count:     55,   Neg. LLF: 881.3538747213574
Optimization terminated successfully    (Exit mode 0)
            Current function value: 881.3538747215401
            Iterations: 10
            Function evaluations: 55
            Gradient evaluations: 10
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1227.934884202285
Iteration:      2,   Func. Count:     13,   Neg. LLF: 889.2892608331276
Iteration:      3,   Func. Count:     19,   Neg. LLF: 888.0315247874007
Iteration:      4,   Func. Count:     26,   Neg. LLF: 884.6907414212512
Iteration:      5,   Func. Count:     31,   Neg. LLF: 884.6848259256694
Iteration:      6,   Func. Count:     36,   Neg. LLF: 884.6574834856478
Iteration:      7,   Func. Count:     41,   Neg. LLF: 884.6397555584947
Iteration:      8,   Func. Count:     46,   Neg. LLF: 884.6286696410642
Iteration:      9,   Func. Count:     51,   Neg. LLF: 884.6273420337402
Iteration:     10,   Func. Count:     56,   Neg. LLF: 884.6273335106139

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:     10,   Func. Count:     55,   Neg. LLF: 895.8111235357464
Optimization terminated successfully    (Exit mode 0)
            Current function value: 895.8111235358674
            Iterations: 10
            Function evaluations: 55
            Gradient evaluations: 10
Iteration:      1,   Func. Count:      6,   Neg. LLF: 902.7968643681711
Iteration:      2,   Func. Count:     12,   Neg. LLF: 938.1897116708617
Iteration:      3,   Func. Count:     19,   Neg. LLF: 902.3366875170718
Iteration:      4,   Func. Count:     26,   Neg. LLF: 898.7667810861799
Iteration:      5,   Func. Count:     32,   Neg. LLF: 898.712149736426
Iteration:      6,   Func. Count:     37,   Neg. LLF: 898.7010577433114
Iteration:      7,   Func. Count:     42,   Neg. LLF: 898.6709540351042
Iteration:      8,   Func. Count:     47,   Neg. LLF: 898.6686370549141
Iteration:      9,   Func. Count:     52,   Neg. LLF: 898.6685609230254
Iteration:     10,   Func. Count:     56,   Neg. LLF: 898.6685609232758


...

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1215.0219667989172
Iteration:      2,   Func. Count:     13,   Neg. LLF: 1014.7641046057527
Iteration:      3,   Func. Count:     19,   Neg. LLF: 1012.9168432390336
Iteration:      4,   Func. Count:     24,   Neg. LLF: 1012.916795479779
Iteration:      5,   Func. Count:     29,   Neg. LLF: 1012.9167683899902
Iteration:      6,   Func. Count:     34,   Neg. LLF: 1012.9167174245744
Iteration:      7,   Func. Count:     39,   Neg. LLF: 1012.9163417193431
Iteration:      8,   Func. Count:     44,   Neg. LLF: 1012.915136468405
Iteration:      9,   Func. Count:     49,   Neg. LLF: 1012.9149662892071
Iteration:     10,   Func. Count:     54,   Neg. LLF: 1012.914940677221
Iteration:     11,   Func. Count:     58,   Neg. LLF: 1012.9149406775548
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1012.914940677221
            Iterations: 11
            Function evaluations: 58
            Gradient evaluat

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1165.6506883972468
Iteration:      2,   Func. Count:     12,   Neg. LLF: 1108.8767328283766
Iteration:      3,   Func. Count:     20,   Neg. LLF: 1049.122615254116
Iteration:      4,   Func. Count:     25,   Neg. LLF: 1053.8626101087798
Iteration:      5,   Func. Count:     31,   Neg. LLF: 1056.5199787652168
Iteration:      6,   Func. Count:     37,   Neg. LLF: 1051.8992846478177
Iteration:      7,   Func. Count:     43,   Neg. LLF: 1050.2432814106528
Iteration:      8,   Func. Count:     49,   Neg. LLF: 1049.4173748869766
Iteration:      9,   Func. Count:     55,   Neg. LLF: 1048.733819232443
Iteration:     10,   Func. Count:     61,   Neg. LLF: 1048.6128017553729
Iteration:     11,   Func. Count:     66,   Neg. LLF: 1048.6103492391446
Iteration:     12,   Func. Count:     71,   Neg. LLF: 1048.6101817179322
Iteration:     13,   Func. Count:     76,   Neg. LLF: 1048.6100240024266
Iteration:     14,   Func. Count:     81,   Neg. LLF:

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Results for the in-sample forecast at 1 month of the naïve GARCH model only including the macro variables:
- AZN-1m-loss: 0.7510213077073029
- BMY-1m-loss: 0.4869672102954541
- JNJ-1m-loss: 0.46835747845563863
- LLY-1m-loss: 0.6937917595311874
- MRK-1m-loss: 0.5864717550381816
- NVO-1m-loss: 0.6669310949370623
- NVS-1m-loss: 0.4589282415210119
- PFE-1m-loss: 0.7735638737687962
- ROG-1m-loss: 3.9891268937734075

### 1.2. 3-step-ahead VaR: In-sample forecast for 3 months

In [17]:
def rolling_3m_forecast(data, dates, ticker, alpha, columns_to_exclude):
    t_loss = 0
    dta = data.copy()

    for d in dates:
        train_df = dta[dta.Date <= d]
        test_df = dta[dta.Date > d]
        test_df = test_df.reset_index(drop=True)

        X_train = train_df.drop(columns=columns_to_exclude)
        y_train = train_df[ticker]

        X_test = test_df.drop(columns=columns_to_exclude)
        X_test = X_test.iloc[0]
        y_test = test_df[ticker]

        garch_model = arch_model(y_train, vol='Garch', p=1, q=1)
        garch_results = garch_model.fit()

        forecasts = garch_results.forecast(start=0, horizon=3)
        cond_vol_forecast = np.sqrt(forecasts.variance.iloc[-1])
        mean_forecast = forecasts.mean.iloc[-1]

        VaR_forecast = mean_forecast + cond_vol_forecast * norm.ppf(alpha)

        loss = tick_loss(alpha, y_test.iloc[0], VaR_forecast)

        t_loss += loss

    t_loss = t_loss / len(dates)

    return t_loss

In [18]:
dates3m = ['2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01', '2022-09-01', '2022-10-01', '2022-11-01', '2022-12-01', '2023-01-01', '2023-02-01']

In [26]:
for t in tickers:
    loss = rolling_3m_forecast(data, dates3m, t, alpha=0.05, columns_to_exclude= to_exclude)
    print(f'{t}-3m-loss: {loss} \n')

Iteration:      1,   Func. Count:      6,   Neg. LLF: 874.1785677557702
Iteration:      2,   Func. Count:     12,   Neg. LLF: 1200.4721531726964
Iteration:      3,   Func. Count:     19,   Neg. LLF: 870.6852289593519
Iteration:      4,   Func. Count:     26,   Neg. LLF: 867.8399619926613
Iteration:      5,   Func. Count:     32,   Neg. LLF: 867.828481407204
Iteration:      6,   Func. Count:     37,   Neg. LLF: 867.8058039900275
Iteration:      7,   Func. Count:     42,   Neg. LLF: 867.748173849943
Iteration:      8,   Func. Count:     47,   Neg. LLF: 867.7462745676646
Iteration:      9,   Func. Count:     52,   Neg. LLF: 867.7461706287772
Iteration:     10,   Func. Count:     57,   Neg. LLF: 867.746169625292
Iteration:     11,   Func. Count:     61,   Neg. LLF: 867.7461696251956
Optimization terminated successfully    (Exit mode 0)
            Current function value: 867.746169625292
            Iterations: 11
            Function evaluations: 61
            Gradient evaluations: 11
It

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1222.9902668269392
Iteration:      2,   Func. Count:     13,   Neg. LLF: 886.8361595084345
Iteration:      3,   Func. Count:     19,   Neg. LLF: 884.5076379612233
Iteration:      4,   Func. Count:     26,   Neg. LLF: 881.4156102171411
Iteration:      5,   Func. Count:     31,   Neg. LLF: 881.4116765176805
Iteration:      6,   Func. Count:     36,   Neg. LLF: 881.3924911968852
Iteration:      7,   Func. Count:     41,   Neg. LLF: 881.3541270434575
Iteration:      8,   Func. Count:     46,   Neg. LLF: 881.353888375146
Iteration:      9,   Func. Count:     51,   Neg. LLF: 881.3538747215401
Iteration:     10,   Func. Count:     55,   Neg. LLF: 881.3538747213574
Optimization terminated successfully    (Exit mode 0)
            Current function value: 881.3538747215401
            Iterations: 10
            Function evaluations: 55
            Gradient evaluations: 10
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1227.934884202285

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      8,   Func. Count:     46,   Neg. LLF: 892.7200619197179
Iteration:      9,   Func. Count:     51,   Neg. LLF: 892.7199879606185
Iteration:     10,   Func. Count:     56,   Neg. LLF: 892.7199864635309
Iteration:     11,   Func. Count:     60,   Neg. LLF: 892.7199864636577
Optimization terminated successfully    (Exit mode 0)
            Current function value: 892.7199864635309
            Iterations: 11
            Function evaluations: 60
            Gradient evaluations: 11
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1244.0857254557427
Iteration:      2,   Func. Count:     13,   Neg. LLF: 900.9392194578554
Iteration:      3,   Func. Count:     19,   Neg. LLF: 898.9513385507807
Iteration:      4,   Func. Count:     26,   Neg. LLF: 895.8637164543424
Iteration:      5,   Func. Count:     31,   Neg. LLF: 895.8604740394894
Iteration:      6,   Func. Count:     36,   Neg. LLF: 895.8453872167669
Iteration:      7,   Func. Count:     41,   Neg. LLF: 895.811571496416

...

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1219.5348841850625
Iteration:      2,   Func. Count:     13,   Neg. LLF: 1018.1340968048978
Iteration:      3,   Func. Count:     19,   Neg. LLF: 1016.2246907221322
Iteration:      4,   Func. Count:     24,   Neg. LLF: 1016.2246771750417
Iteration:      5,   Func. Count:     29,   Neg. LLF: 1016.2246658805437
Iteration:      6,   Func. Count:     34,   Neg. LLF: 1016.2246643659951
Iteration:      7,   Func. Count:     38,   Neg. LLF: 1016.2246643664057
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1016.2246643659951
            Iterations: 7
            Function evaluations: 38
            Gradient evaluations: 7
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1224.3227681527878
Iteration:      2,   Func. Count:     13,   Neg. LLF: 1021.7568942469534
Iteration:      3,   Func. Count:     19,   Neg. LLF: 1019.8138887813601
Iteration:      4,   Func. Count:     24,   Neg. LLF: 1019.813

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1233.4593893444899
Iteration:      2,   Func. Count:     13,   Neg. LLF: 1028.629179221663
Iteration:      3,   Func. Count:     19,   Neg. LLF: 1026.5330869611244
Iteration:      4,   Func. Count:     24,   Neg. LLF: 1026.5334575056986
Iteration:      5,   Func. Count:     30,   Neg. LLF: 1026.5321650488156
Iteration:      6,   Func. Count:     35,   Neg. LLF: 1026.5321236507464
Iteration:      7,   Func. Count:     40,   Neg. LLF: 1026.531915967955
Iteration:      8,   Func. Count:     45,   Neg. LLF: 1026.5310091930328
Iteration:      9,   Func. Count:     50,   Neg. LLF: 1026.5286922601586
Iteration:     10,   Func. Count:     55,   Neg. LLF: 1026.5282609248545
Iteration:     11,   Func. Count:     60,   Neg. LLF: 1026.5282287876234
Iteration:     12,   Func. Count:     65,   Neg. LLF: 1026.528228302634
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1026.528228302634
            Iterati

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Results for the in-sample forecast at 3 months of the naïve GARCH model only including the macro variables:
- AZN-3m-loss: 2.283164388937413
- BMY-3m-loss: 1.5404835438780882
- JNJ-3m-loss: 1.4817182156856672
- LLY-3m-loss: 1.7815904402959126
- MRK-3m-loss: 1.747261002195295
- NVO-3m-loss: 2.041623352733811
- NVS-3m-loss: 1.2084892475952937
- PFE-3m-loss: 2.5518787621946375
- ROG-3m-loss: 13.757700842141588

### 1.3. 6-step-ahead VaR: In-sample forecast for 6 months

In [19]:
def rolling_6m_forecast(data, dates, ticker, alpha, columns_to_exclude):
    t_loss = 0
    dta = data.copy()

    for d in dates:
        train_df = dta[dta.Date <= d]
        test_df = dta[dta.Date > d]
        test_df = test_df.reset_index(drop=True)

        X_train = train_df.drop(columns=columns_to_exclude)
        y_train = train_df[ticker]

        X_test = test_df.drop(columns=columns_to_exclude)
        X_test = X_test.iloc[0]
        y_test = test_df[ticker]

        garch_model = arch_model(y_train, vol='Garch', p=1, q=1)
        garch_results = garch_model.fit()

        forecasts = garch_results.forecast(start=0, horizon=6)
        cond_vol_forecast = np.sqrt(forecasts.variance.iloc[-1])
        mean_forecast = forecasts.mean.iloc[-1]

        VaR_forecast = mean_forecast + cond_vol_forecast * norm.ppf(alpha)

        loss = tick_loss(alpha, y_test.iloc[0], VaR_forecast)

        t_loss += loss

    t_loss = t_loss / len(dates)

    return t_loss

In [20]:
dates6m = ['2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01', '2022-09-01', '2022-10-01', '2022-11-01']

In [29]:
for t in tickers:
    loss = rolling_6m_forecast(data, dates6m, t, alpha=0.05, columns_to_exclude= to_exclude)
    print(f'{t}-6m-loss: {loss} \n')

Iteration:      1,   Func. Count:      6,   Neg. LLF: 874.1785677557702
Iteration:      2,   Func. Count:     12,   Neg. LLF: 1200.4721531726964
Iteration:      3,   Func. Count:     19,   Neg. LLF: 870.6852289593519
Iteration:      4,   Func. Count:     26,   Neg. LLF: 867.8399619926613
Iteration:      5,   Func. Count:     32,   Neg. LLF: 867.828481407204
Iteration:      6,   Func. Count:     37,   Neg. LLF: 867.8058039900275
Iteration:      7,   Func. Count:     42,   Neg. LLF: 867.748173849943
Iteration:      8,   Func. Count:     47,   Neg. LLF: 867.7462745676646
Iteration:      9,   Func. Count:     52,   Neg. LLF: 867.7461706287772
Iteration:     10,   Func. Count:     57,   Neg. LLF: 867.746169625292
Iteration:     11,   Func. Count:     61,   Neg. LLF: 867.7461696251956
Optimization terminated successfully    (Exit mode 0)
            Current function value: 867.746169625292
            Iterations: 11
            Function evaluations: 61
            Gradient evaluations: 11
It

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1222.9902668269392
Iteration:      2,   Func. Count:     13,   Neg. LLF: 886.8361595084345
Iteration:      3,   Func. Count:     19,   Neg. LLF: 884.5076379612233
Iteration:      4,   Func. Count:     26,   Neg. LLF: 881.4156102171411
Iteration:      5,   Func. Count:     31,   Neg. LLF: 881.4116765176805
Iteration:      6,   Func. Count:     36,   Neg. LLF: 881.3924911968852
Iteration:      7,   Func. Count:     41,   Neg. LLF: 881.3541270434575
Iteration:      8,   Func. Count:     46,   Neg. LLF: 881.353888375146
Iteration:      9,   Func. Count:     51,   Neg. LLF: 881.3538747215401
Iteration:     10,   Func. Count:     55,   Neg. LLF: 881.3538747213574
Optimization terminated successfully    (Exit mode 0)
            Current function value: 881.3538747215401
            Iterations: 10
            Function evaluations: 55
            Gradient evaluations: 10
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1227.934884202285

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      1,   Func. Count:      6,   Neg. LLF: 906.9964907219263
Iteration:      2,   Func. Count:     13,   Neg. LLF: 913.222709191811
Iteration:      3,   Func. Count:     19,   Neg. LLF: 904.0742102389393
Iteration:      4,   Func. Count:     26,   Neg. LLF: 888.0176227402903
Iteration:      5,   Func. Count:     31,   Neg. LLF: 888.0170831290263
Iteration:      6,   Func. Count:     36,   Neg. LLF: 888.0170406502615
Iteration:      7,   Func. Count:     41,   Neg. LLF: 888.0167817047684
Iteration:      8,   Func. Count:     46,   Neg. LLF: 888.0154865930697
Iteration:      9,   Func. Count:     51,   Neg. LLF: 888.0109869234884
Iteration:     10,   Func. Count:     56,   Neg. LLF: 888.009594587629
Iteration:     11,   Func. Count:     61,   Neg. LLF: 888.0095017604818
Iteration:     12,   Func. Count:     66,   Neg. LLF: 888.0094967912892
Iteration:     13,   Func. Count:     70,   Neg. LLF: 888.0094967911114
Optimization terminated successfully    (Exit mode 0)
            

...

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:     10,   Func. Count:     54,   Neg. LLF: 1012.914940677221
Iteration:     11,   Func. Count:     58,   Neg. LLF: 1012.9149406775548
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1012.914940677221
            Iterations: 11
            Function evaluations: 58
            Gradient evaluations: 11
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1219.5348841850625
Iteration:      2,   Func. Count:     13,   Neg. LLF: 1018.1340968048978
Iteration:      3,   Func. Count:     19,   Neg. LLF: 1016.2246907221322
Iteration:      4,   Func. Count:     24,   Neg. LLF: 1016.2246771750417
Iteration:      5,   Func. Count:     29,   Neg. LLF: 1016.2246658805437
Iteration:      6,   Func. Count:     34,   Neg. LLF: 1016.2246643659951
Iteration:      7,   Func. Count:     38,   Neg. LLF: 1016.2246643664057
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1016.2246643659951
            Iterations: 7


/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1233.4593893444899
Iteration:      2,   Func. Count:     13,   Neg. LLF: 1028.629179221663
Iteration:      3,   Func. Count:     19,   Neg. LLF: 1026.5330869611244
Iteration:      4,   Func. Count:     24,   Neg. LLF: 1026.5334575056986
Iteration:      5,   Func. Count:     30,   Neg. LLF: 1026.5321650488156
Iteration:      6,   Func. Count:     35,   Neg. LLF: 1026.5321236507464
Iteration:      7,   Func. Count:     40,   Neg. LLF: 1026.531915967955
Iteration:      8,   Func. Count:     45,   Neg. LLF: 1026.5310091930328
Iteration:      9,   Func. Count:     50,   Neg. LLF: 1026.5286922601586
Iteration:     10,   Func. Count:     55,   Neg. LLF: 1026.5282609248545
Iteration:     11,   Func. Count:     60,   Neg. LLF: 1026.5282287876234
Iteration:     12,   Func. Count:     65,   Neg. LLF: 1026.528228302634
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1026.528228302634
            Iterati

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


Results for the in-sample forecast at 6 months of the naïve GARCH model only including the macro variables:
- AZN-6m-loss: 4.903444131504825
- BMY-6m-loss: 3.145314758733963
- JNJ-6m-loss: 2.9673583449333343
- LLY-6m-loss: 3.955517166440173
- MRK-6m-loss: 3.8533075712325933
- NVO-6m-loss: 3.8323239117351013
- NVS-6m-loss: 2.337982438881043
- PFE-6m-loss: 3.47848250024052
- ROG-6m-loss: 35.04284232898197

### 1.4. 9-step-ahead VaR: In-sample forecast for 9 months

In [21]:
def rolling_9m_forecast(data, dates, ticker, alpha, columns_to_exclude):
    t_loss = 0
    dta = data.copy()

    for d in dates:
        train_df = dta[dta.Date <= d]
        test_df = dta[dta.Date > d]
        test_df = test_df.reset_index(drop=True)

        X_train = train_df.drop(columns=columns_to_exclude)
        y_train = train_df[ticker]

        X_test = test_df.drop(columns=columns_to_exclude)
        X_test = X_test.iloc[0]
        y_test = test_df[ticker]

        garch_model = arch_model(y_train, vol='Garch', p=1, q=1)
        garch_results = garch_model.fit()

        forecasts = garch_results.forecast(start=0, horizon=9)
        cond_vol_forecast = np.sqrt(forecasts.variance.iloc[-1])
        mean_forecast = forecasts.mean.iloc[-1]

        VaR_forecast = mean_forecast + cond_vol_forecast * norm.ppf(alpha)

        loss = tick_loss(alpha, y_test.iloc[0], VaR_forecast)

        t_loss += loss

    t_loss = t_loss / len(dates)

    return t_loss

In [22]:
dates9m = ['2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01']

In [32]:
for t in tickers:
    loss = rolling_9m_forecast(data, dates9m, t, alpha=0.05, columns_to_exclude= to_exclude)
    print(f'{t}-9m-loss: {loss} \n')

Iteration:      1,   Func. Count:      6,   Neg. LLF: 874.1785677557702
Iteration:      2,   Func. Count:     12,   Neg. LLF: 1200.4721531726964
Iteration:      3,   Func. Count:     19,   Neg. LLF: 870.6852289593519
Iteration:      4,   Func. Count:     26,   Neg. LLF: 867.8399619926613
Iteration:      5,   Func. Count:     32,   Neg. LLF: 867.828481407204
Iteration:      6,   Func. Count:     37,   Neg. LLF: 867.8058039900275
Iteration:      7,   Func. Count:     42,   Neg. LLF: 867.748173849943
Iteration:      8,   Func. Count:     47,   Neg. LLF: 867.7462745676646
Iteration:      9,   Func. Count:     52,   Neg. LLF: 867.7461706287772
Iteration:     10,   Func. Count:     57,   Neg. LLF: 867.746169625292
Iteration:     11,   Func. Count:     61,   Neg. LLF: 867.7461696251956
Optimization terminated successfully    (Exit mode 0)
            Current function value: 867.746169625292
            Iterations: 11
            Function evaluations: 61
            Gradient evaluations: 11
It

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      3,   Func. Count:     20,   Neg. LLF: 878.1339497577962
Iteration:      4,   Func. Count:     27,   Neg. LLF: 876.6737053132299
Iteration:      5,   Func. Count:     32,   Neg. LLF: 876.6672585066991
Iteration:      6,   Func. Count:     37,   Neg. LLF: 876.6384410362984
Iteration:      7,   Func. Count:     42,   Neg. LLF: 876.589180523503
Iteration:      8,   Func. Count:     47,   Neg. LLF: 876.587517374675
Iteration:      9,   Func. Count:     52,   Neg. LLF: 876.5873820200592
Iteration:     10,   Func. Count:     57,   Neg. LLF: 876.5873810789516
Optimization terminated successfully    (Exit mode 0)
            Current function value: 876.5873810789516
            Iterations: 10
            Function evaluations: 57
            Gradient evaluations: 10
AZN-9m-loss: 6.615671740041865 

Iteration:      1,   Func. Count:      6,   Neg. LLF: 904.2346809763922
Iteration:      2,   Func. Count:     13,   Neg. LLF: 909.8940809007032
Iteration:      3,   Func. Count:     19

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      1,   Func. Count:      6,   Neg. LLF: 909.9915114450789
Iteration:      2,   Func. Count:     13,   Neg. LLF: 916.2161491790237
Iteration:      3,   Func. Count:     19,   Neg. LLF: 906.9161318222093
Iteration:      4,   Func. Count:     26,   Neg. LLF: 891.0219297504891
Iteration:      5,   Func. Count:     31,   Neg. LLF: 891.0213654327135
Iteration:      6,   Func. Count:     36,   Neg. LLF: 891.0213275042972
Iteration:      7,   Func. Count:     41,   Neg. LLF: 891.0210963483075
Iteration:      8,   Func. Count:     46,   Neg. LLF: 891.0199400989616
Iteration:      9,   Func. Count:     51,   Neg. LLF: 891.0159586622088
Iteration:     10,   Func. Count:     56,   Neg. LLF: 891.0147657259877
Iteration:     11,   Func. Count:     61,   Neg. LLF: 891.014698196681
Iteration:     12,   Func. Count:     66,   Neg. LLF: 891.0146939301426
Iteration:     13,   Func. Count:     70,   Neg. LLF: 891.0146939300045
Optimization terminated successfully    (Exit mode 0)
           

...

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1058.3131753399266
Iteration:      2,   Func. Count:     13,   Neg. LLF: 857.7749864560318
Iteration:      3,   Func. Count:     20,   Neg. LLF: 962.5766427766032
Iteration:      4,   Func. Count:     27,   Neg. LLF: 843.349449512671
Iteration:      5,   Func. Count:     32,   Neg. LLF: 843.2718780137172
Iteration:      6,   Func. Count:     37,   Neg. LLF: 843.368911001076
Iteration:      7,   Func. Count:     43,   Neg. LLF: 843.2570399235453
Iteration:      8,   Func. Count:     48,   Neg. LLF: 843.256746432
Iteration:      9,   Func. Count:     53,   Neg. LLF: 843.2567433992162
Iteration:     10,   Func. Count:     57,   Neg. LLF: 843.256743398649
Optimization terminated successfully    (Exit mode 0)
            Current function value: 843.2567433992162
            Iterations: 10
            Function evaluations: 57
            Gradient evaluations: 10
PFE-9m-loss: 3.4260101010202124 

Iteration:      1,   Func. Count:      6,  

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1219.5348841850625
Iteration:      2,   Func. Count:     13,   Neg. LLF: 1018.1340968048978
Iteration:      3,   Func. Count:     19,   Neg. LLF: 1016.2246907221322
Iteration:      4,   Func. Count:     24,   Neg. LLF: 1016.2246771750417
Iteration:      5,   Func. Count:     29,   Neg. LLF: 1016.2246658805437
Iteration:      6,   Func. Count:     34,   Neg. LLF: 1016.2246643659951
Iteration:      7,   Func. Count:     38,   Neg. LLF: 1016.2246643664057
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1016.2246643659951
            Iterations: 7
            Function evaluations: 38
            Gradient evaluations: 7
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1224.3227681527878
Iteration:      2,   Func. Count:     13,   Neg. LLF: 1021.7568942469534
Iteration:      3,   Func. Count:     19,   Neg. LLF: 1019.8138887813601
Iteration:      4,   Func. Count:     24,   Neg. LLF: 1019.813

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


Results for the in-sample forecast at 9 months of the naïve GARCH model only including the macro variables:
- AZN-9m-loss: 6.615671740041865
- BMY-9m-loss: 4.273425518571841
- JNJ-9m-loss: 4.609001006719325
- LLY-9m-loss: 5.27799903274221
- MRK-9m-loss: 3.594498717303232
- NVO-9m-loss: 3.1426340253078324
- NVS-9m-loss: 1.3872351248346997
- PFE-9m-loss: 3.4260101010202124
- ROG-9m-loss: 6.497664225334868

### 1.5. 12-step-ahead VaR: In-sample forecast for 12 months

In [23]:
def rolling_12m_forecast(data, dates, ticker, alpha, columns_to_exclude):
    t_loss = 0
    dta = data.copy()

    for d in dates:
        train_df = dta[dta.Date <= d]
        test_df = dta[dta.Date > d]
        test_df = test_df.reset_index(drop=True)

        X_train = train_df.drop(columns=columns_to_exclude)
        y_train = train_df[ticker]

        X_test = test_df.drop(columns=columns_to_exclude)
        X_test = X_test.iloc[0]
        y_test = test_df[ticker]

        garch_model = arch_model(y_train, vol='Garch', p=1, q=1)
        garch_results = garch_model.fit()

        forecasts = garch_results.forecast(start=0, horizon=12)
        cond_vol_forecast = np.sqrt(forecasts.variance.iloc[-1])
        mean_forecast = forecasts.mean.iloc[-1]

        VaR_forecast = mean_forecast + cond_vol_forecast * norm.ppf(alpha)

        loss = tick_loss(alpha, y_test.iloc[0], VaR_forecast)

        t_loss += loss

    t_loss = t_loss / len(dates)

    return t_loss

In [24]:
dates12m = ['2022-05-01']

In [35]:
for t in tickers:
    loss = rolling_12m_forecast(data, dates9m, t, alpha=0.05, columns_to_exclude= to_exclude)
    print(f'{t}-12m-loss: {loss} \n')

Iteration:      1,   Func. Count:      6,   Neg. LLF: 874.1785677557702
Iteration:      2,   Func. Count:     12,   Neg. LLF: 1200.4721531726964
Iteration:      3,   Func. Count:     19,   Neg. LLF: 870.6852289593519
Iteration:      4,   Func. Count:     26,   Neg. LLF: 867.8399619926613
Iteration:      5,   Func. Count:     32,   Neg. LLF: 867.828481407204
Iteration:      6,   Func. Count:     37,   Neg. LLF: 867.8058039900275
Iteration:      7,   Func. Count:     42,   Neg. LLF: 867.748173849943
Iteration:      8,   Func. Count:     47,   Neg. LLF: 867.7462745676646
Iteration:      9,   Func. Count:     52,   Neg. LLF: 867.7461706287772
Iteration:     10,   Func. Count:     57,   Neg. LLF: 867.746169625292
Iteration:     11,   Func. Count:     61,   Neg. LLF: 867.7461696251956
Optimization terminated successfully    (Exit mode 0)
            Current function value: 867.746169625292
            Iterations: 11
            Function evaluations: 61
            Gradient evaluations: 11
It

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


Iteration:      1,   Func. Count:      6,   Neg. LLF: 1029.2218302268182
Iteration:      2,   Func. Count:     14,   Neg. LLF: 876.677417077776
Iteration:      3,   Func. Count:     20,   Neg. LLF: 893.4391948088039
Iteration:      4,   Func. Count:     27,   Neg. LLF: 873.3205489314121
Iteration:      5,   Func. Count:     32,   Neg. LLF: 873.30676887837
Iteration:      6,   Func. Count:     37,   Neg. LLF: 873.2807082481356
Iteration:      7,   Func. Count:     42,   Neg. LLF: 873.2537478418432
Iteration:      8,   Func. Count:     47,   Neg. LLF: 873.2366633570832
Iteration:      9,   Func. Count:     52,   Neg. LLF: 873.2321873867108
Iteration:     10,   Func. Count:     57,   Neg. LLF: 873.2321447169634
Iteration:     11,   Func. Count:     61,   Neg. LLF: 873.2321447170441
Optimization terminated successfully    (Exit mode 0)
            Current function value: 873.2321447169634
            Iterations: 11
            Function evaluations: 61
            Gradient evaluations: 11
I

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


AZN-12m-loss: 8.682244208869232 

Iteration:      1,   Func. Count:      6,   Neg. LLF: 904.2346809763922
Iteration:      2,   Func. Count:     13,   Neg. LLF: 909.8940809007032
Iteration:      3,   Func. Count:     19,   Neg. LLF: 901.9676011216195
Iteration:      4,   Func. Count:     26,   Neg. LLF: 885.2200651421106
Iteration:      5,   Func. Count:     31,   Neg. LLF: 885.2195442810739
Iteration:      6,   Func. Count:     36,   Neg. LLF: 885.2195104425602
Iteration:      7,   Func. Count:     41,   Neg. LLF: 885.2193138255832
Iteration:      8,   Func. Count:     46,   Neg. LLF: 885.2183522300605
Iteration:      9,   Func. Count:     51,   Neg. LLF: 885.2147616595502
Iteration:     10,   Func. Count:     56,   Neg. LLF: 885.2128950597746
Iteration:     11,   Func. Count:     61,   Neg. LLF: 885.2127609855501
Iteration:     12,   Func. Count:     66,   Neg. LLF: 885.2127585731427
Iteration:     13,   Func. Count:     70,   Neg. LLF: 885.2127585730582
Optimization terminated succes

...

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1058.3131753399266
Iteration:      2,   Func. Count:     13,   Neg. LLF: 857.7749864560318
Iteration:      3,   Func. Count:     20,   Neg. LLF: 962.5766427766032
Iteration:      4,   Func. Count:     27,   Neg. LLF: 843.349449512671
Iteration:      5,   Func. Count:     32,   Neg. LLF: 843.2718780137172
Iteration:      6,   Func. Count:     37,   Neg. LLF: 843.368911001076
Iteration:      7,   Func. Count:     43,   Neg. LLF: 843.2570399235453
Iteration:      8,   Func. Count:     48,   Neg. LLF: 843.256746432
Iteration:      9,   Func. Count:     53,   Neg. LLF: 843.2567433992162
Iteration:     10,   Func. Count:     57,   Neg. LLF: 843.256743398649
Optimization terminated successfully    (Exit mode 0)
            Current function value: 843.2567433992162
            Iterations: 10
            Function evaluations: 57
            Gradient evaluations: 10
PFE-12m-loss: 4.463282208698661 

Iteration:      1,   Func. Count:      6,  

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


Iteration:      5,   Func. Count:     29,   Neg. LLF: 1019.8138374640391
Iteration:      6,   Func. Count:     34,   Neg. LLF: 1019.8138352299792
Iteration:      7,   Func. Count:     39,   Neg. LLF: 1019.8138339417283
Iteration:      8,   Func. Count:     44,   Neg. LLF: 1019.8138256839918
Iteration:      9,   Func. Count:     49,   Neg. LLF: 1019.8137889992914
Iteration:     10,   Func. Count:     54,   Neg. LLF: 1019.8137575943713
Iteration:     11,   Func. Count:     59,   Neg. LLF: 1019.8137503012683
Iteration:     12,   Func. Count:     64,   Neg. LLF: 1019.8137489296873
Iteration:     13,   Func. Count:     68,   Neg. LLF: 1019.813748930081
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1019.8137489296873
            Iterations: 13
            Function evaluations: 68
            Gradient evaluations: 13
ROG-12m-loss: 8.710335288147625 



/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


Results for the in-sample forecast at 12 months of the naïve GARCH model only including the macro variables:
- AZN-12m-loss: 8.682244208869232
- BMY-12m-loss: 5.718378515719505
- JNJ-12m-loss: 6.167350824508993
- LLY-12m-loss: 6.9713916616637395
- MRK-12m-loss: 4.788628365010815
- NVO-12m-loss: 4.192077007602673
- NVS-12m-loss: 1.850423734249277
- PFE-12m-loss: 4.463282208698661
- ROG-12m-loss: 8.710335288147625

## 2. Ticker Data

In [25]:
data_stocks = data.copy()
to_exclude2 = []

for covariate in covs:
    to_exclude2.append(covariate)
    to_exclude2.append(f'{covariate}_lag1')
    to_exclude2.append(f'{covariate}_lag2')
    to_exclude2.append(f'{covariate}_lag3')

for t in tickers:
    to_exclude2.append(t)

to_exclude2.append('Date')

In [27]:
to_exclude2

['unemployment_rate',
 'unemployment_rate_lag1',
 'unemployment_rate_lag2',
 'unemployment_rate_lag3',
 'CPI',
 'CPI_lag1',
 'CPI_lag2',
 'CPI_lag3',
 'treasury_yield',
 'treasury_yield_lag1',
 'treasury_yield_lag2',
 'treasury_yield_lag3',
 'GDP_growth',
 'GDP_growth_lag1',
 'GDP_growth_lag2',
 'GDP_growth_lag3',
 'SP500_return',
 'SP500_return_lag1',
 'SP500_return_lag2',
 'SP500_return_lag3',
 'inflation_change',
 'inflation_change_lag1',
 'inflation_change_lag2',
 'inflation_change_lag3',
 'unemp_change',
 'unemp_change_lag1',
 'unemp_change_lag2',
 'unemp_change_lag3',
 'treasury_yield_change',
 'treasury_yield_change_lag1',
 'treasury_yield_change_lag2',
 'treasury_yield_change_lag3',
 'AZN',
 'BMY',
 'JNJ',
 'LLY',
 'MRK',
 'NVO',
 'NVS',
 'PFE',
 'ROG',
 'Date']

### 2.1. 1-step-ahead VaR: In-sample forecast for 1 month

In [38]:
for t in tickers:
    loss = rolling_1m_forecast(data, dates1m, t, alpha=0.05, columns_to_exclude= to_exclude2)
    print(f'{t}-1m-loss: {loss} \n')

Iteration:      1,   Func. Count:      6,   Neg. LLF: 874.1785677557702
Iteration:      2,   Func. Count:     12,   Neg. LLF: 1200.4721531726964
Iteration:      3,   Func. Count:     19,   Neg. LLF: 870.6852289593519
Iteration:      4,   Func. Count:     26,   Neg. LLF: 867.8399619926613
Iteration:      5,   Func. Count:     32,   Neg. LLF: 867.828481407204
Iteration:      6,   Func. Count:     37,   Neg. LLF: 867.8058039900275
Iteration:      7,   Func. Count:     42,   Neg. LLF: 867.748173849943
Iteration:      8,   Func. Count:     47,   Neg. LLF: 867.7462745676646
Iteration:      9,   Func. Count:     52,   Neg. LLF: 867.7461706287772
Iteration:     10,   Func. Count:     57,   Neg. LLF: 867.746169625292
Iteration:     11,   Func. Count:     61,   Neg. LLF: 867.7461696251956
Optimization terminated successfully    (Exit mode 0)
            Current function value: 867.746169625292
            Iterations: 11
            Function evaluations: 61
            Gradient evaluations: 11
It

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      1,   Func. Count:      6,   Neg. LLF: 947.3225299769143
Iteration:      2,   Func. Count:     14,   Neg. LLF: 880.899936039578
Iteration:      3,   Func. Count:     20,   Neg. LLF: 878.1339497577962
Iteration:      4,   Func. Count:     27,   Neg. LLF: 876.6737053132299
Iteration:      5,   Func. Count:     32,   Neg. LLF: 876.6672585066991
Iteration:      6,   Func. Count:     37,   Neg. LLF: 876.6384410362984
Iteration:      7,   Func. Count:     42,   Neg. LLF: 876.589180523503
Iteration:      8,   Func. Count:     47,   Neg. LLF: 876.587517374675
Iteration:      9,   Func. Count:     52,   Neg. LLF: 876.5873820200592
Iteration:     10,   Func. Count:     57,   Neg. LLF: 876.5873810789516
Optimization terminated successfully    (Exit mode 0)
            Current function value: 876.5873810789516
            Iterations: 10
            Function evaluations: 57
            Gradient evaluations: 10
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1222.9902668269392
I

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1178.7314788173323
Iteration:      2,   Func. Count:     14,   Neg. LLF: 893.9662722206108
Iteration:      3,   Func. Count:     21,   Neg. LLF: 947.2792115942004
Iteration:      4,   Func. Count:     28,   Neg. LLF: 889.7749000463948
Iteration:      5,   Func. Count:     33,   Neg. LLF: 889.7675789803586
Iteration:      6,   Func. Count:     38,   Neg. LLF: 889.7537238994715
Iteration:      7,   Func. Count:     43,   Neg. LLF: 889.7361309667915
Iteration:      8,   Func. Count:     48,   Neg. LLF: 889.7274260931832
Iteration:      9,   Func. Count:     53,   Neg. LLF: 889.725416391847
Iteration:     10,   Func. Count:     58,   Neg. LLF: 889.7253145862583
Iteration:     11,   Func. Count:     62,   Neg. LLF: 889.7253145862426
Optimization terminated successfully    (Exit mode 0)
            Current function value: 889.7253145862583
            Iterations: 11
            Function evaluations: 62
            Gradient evaluations: 11

...

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

 68
            Gradient evaluations: 13
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1228.9085794123553
Iteration:      2,   Func. Count:     13,   Neg. LLF: 1025.2056303614033
Iteration:      3,   Func. Count:     19,   Neg. LLF: 1023.1943328513626
Iteration:      4,   Func. Count:     24,   Neg. LLF: 1023.1942387003072
Iteration:      5,   Func. Count:     29,   Neg. LLF: 1023.1941278027645
Iteration:      6,   Func. Count:     34,   Neg. LLF: 1023.1941125884837
Iteration:      7,   Func. Count:     39,   Neg. LLF: 1023.1940196342322
Iteration:      8,   Func. Count:     44,   Neg. LLF: 1023.1938083986237
Iteration:      9,   Func. Count:     49,   Neg. LLF: 1023.1934615938222
Iteration:     10,   Func. Count:     54,   Neg. LLF: 1023.1930778873815
Iteration:     11,   Func. Count:     59,   Neg. LLF: 1023.1928755673607
Iteration:     12,   Func. Count:     64,   Neg. LLF: 1023.1928461598402
Iteration:     13,   Func. Count:     69,   Neg. LLF: 1023.1928437372644
Iteration:

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1161.1001574452678
Iteration:      2,   Func. Count:     12,   Neg. LLF: 1087.735651160859
Iteration:      3,   Func. Count:     20,   Neg. LLF: 1044.883827289639
Iteration:      4,   Func. Count:     25,   Neg. LLF: 1046.7587090974273
Iteration:      5,   Func. Count:     31,   Neg. LLF: 1050.487245194426
Iteration:      6,   Func. Count:     37,   Neg. LLF: 1046.482878586167
Iteration:      7,   Func. Count:     43,   Neg. LLF: 1045.429020908597
Iteration:      8,   Func. Count:     49,   Neg. LLF: 1044.8013650643516
Iteration:      9,   Func. Count:     55,   Neg. LLF: 1044.4090513042167
Iteration:     10,   Func. Count:     61,   Neg. LLF: 1044.3920663986553
Iteration:     11,   Func. Count:     66,   Neg. LLF: 1044.3919030758052
Iteration:     12,   Func. Count:     71,   Neg. LLF: 1044.3918989609401
Iteration:     13,   Func. Count:     76,   Neg. LLF: 1044.3918984136199
Optimization terminated successfully    (Exit mode 0)
  

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Results for the in-sample forecast at 1 month of the naïve GARCH model only including the stock related variables:
- AZN-1m-loss: 0.7510213077073029
- BMY-1m-loss: 0.4869672102954541
- JNJ-1m-loss: 0.46835747845563863
- LLY-1m-loss: 0.6937917595311874
- MRK-1m-loss: 0.5864717550381816
- NVO-1m-loss: 0.6669310949370623
- NVS-1m-loss: 0.4589282415210119
- PFE-1m-loss: 0.7735638737687962
- ROG-1m-loss: 3.9891268937734075

### 2.2. 3-step-ahead VaR: In-sample forecast for 3 months

In [39]:
for t in tickers:
    loss = rolling_3m_forecast(data, dates3m, t, alpha=0.05, columns_to_exclude= to_exclude2)
    print(f'{t}-3m-loss: {loss} \n')

Iteration:      1,   Func. Count:      6,   Neg. LLF: 874.1785677557702
Iteration:      2,   Func. Count:     12,   Neg. LLF: 1200.4721531726964
Iteration:      3,   Func. Count:     19,   Neg. LLF: 870.6852289593519
Iteration:      4,   Func. Count:     26,   Neg. LLF: 867.8399619926613
Iteration:      5,   Func. Count:     32,   Neg. LLF: 867.828481407204
Iteration:      6,   Func. Count:     37,   Neg. LLF: 867.8058039900275
Iteration:      7,   Func. Count:     42,   Neg. LLF: 867.748173849943
Iteration:      8,   Func. Count:     47,   Neg. LLF: 867.7462745676646
Iteration:      9,   Func. Count:     52,   Neg. LLF: 867.7461706287772
Iteration:     10,   Func. Count:     57,   Neg. LLF: 867.746169625292
Iteration:     11,   Func. Count:     61,   Neg. LLF: 867.7461696251956
Optimization terminated successfully    (Exit mode 0)
            Current function value: 867.746169625292
            Iterations: 11
            Function evaluations: 61
            Gradient evaluations: 11
It

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


Iteration:      6,   Func. Count:     38,   Neg. LLF: 870.5633112349748
Iteration:      7,   Func. Count:     43,   Neg. LLF: 870.5202826921204
Iteration:      8,   Func. Count:     48,   Neg. LLF: 870.5162024038798
Iteration:      9,   Func. Count:     53,   Neg. LLF: 870.5158971858509
Iteration:     10,   Func. Count:     58,   Neg. LLF: 870.5158945251494
Iteration:     11,   Func. Count:     62,   Neg. LLF: 870.5158945255108
Optimization terminated successfully    (Exit mode 0)
            Current function value: 870.5158945251494
            Iterations: 11
            Function evaluations: 62
            Gradient evaluations: 11
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1029.2218302268182
Iteration:      2,   Func. Count:     14,   Neg. LLF: 876.677417077776
Iteration:      3,   Func. Count:     20,   Neg. LLF: 893.4391948088039
Iteration:      4,   Func. Count:     27,   Neg. LLF: 873.3205489314121
Iteration:      5,   Func. Count:     32,   Neg. LLF: 873.30676887837
I

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(



Iteration:      5,   Func. Count:     32,   Neg. LLF: 876.6672585066991
Iteration:      6,   Func. Count:     37,   Neg. LLF: 876.6384410362984
Iteration:      7,   Func. Count:     42,   Neg. LLF: 876.589180523503
Iteration:      8,   Func. Count:     47,   Neg. LLF: 876.587517374675
Iteration:      9,   Func. Count:     52,   Neg. LLF: 876.5873820200592
Iteration:     10,   Func. Count:     57,   Neg. LLF: 876.5873810789516
Optimization terminated successfully    (Exit mode 0)
            Current function value: 876.5873810789516
            Iterations: 10
            Function evaluations: 57
            Gradient evaluations: 10
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1222.9902668269392
Iteration:      2,   Func. Count:     13,   Neg. LLF: 886.8361595084345
Iteration:      3,   Func. Count:     19,   Neg. LLF: 884.5076379612233
Iteration:      4,   Func. Count:     26,   Neg. LLF: 881.4156102171411
Iteration:      5,   Func. Count:     31,   Neg. LLF: 881.4116765176805

...

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1219.5348841850625
Iteration:      2,   Func. Count:     13,   Neg. LLF: 1018.1340968048978
Iteration:      3,   Func. Count:     19,   Neg. LLF: 1016.2246907221322
Iteration:      4,   Func. Count:     24,   Neg. LLF: 1016.2246771750417
Iteration:      5,   Func. Count:     29,   Neg. LLF: 1016.2246658805437
Iteration:      6,   Func. Count:     34,   Neg. LLF: 1016.2246643659951
Iteration:      7,   Func. Count:     38,   Neg. LLF: 1016.2246643664057
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1016.2246643659951
            Iterations: 7
            Function evaluations: 38
            Gradient evaluations: 7
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1224.3227681527878
Iteration:      2,   Func. Count:     13,   Neg. LLF: 1021.7568942469534
Iteration:      3,   Func. Count:     19,   Neg. LLF: 1019.8138887813601
Iteration:      4,   Func. Count:     24,   Neg. LLF: 1019.813

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:     10,   Func. Count:     55,   Neg. LLF: 1026.5282609248545
Iteration:     11,   Func. Count:     60,   Neg. LLF: 1026.5282287876234
Iteration:     12,   Func. Count:     65,   Neg. LLF: 1026.528228302634
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1026.528228302634
            Iterations: 12
            Function evaluations: 65
            Gradient evaluations: 12
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1156.7999856927881
Iteration:      2,   Func. Count:     12,   Neg. LLF: 1075.3769103078384
Iteration:      3,   Func. Count:     20,   Neg. LLF: 1041.1938827489212
Iteration:      4,   Func. Count:     25,   Neg. LLF: 1050.0008570604436
Iteration:      5,   Func. Count:     31,   Neg. LLF: 1048.2288453655756
Iteration:      6,   Func. Count:     37,   Neg. LLF: 1043.837904983452
Iteration:      7,   Func. Count:     43,   Neg. LLF: 1042.206502283325
Iteration:      8,   Func. Count:     49,   Neg. LLF: 1041.41407

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Results for the in-sample forecast at 3 months of the naïve GARCH model only including the stock related variables:
- AZN-3m-loss: 2.283164388937413
- BMY-3m-loss: 1.5404835438780882
- JNJ-3m-loss: 1.4817182156856672
- LLY-3m-loss: 1.7815904402959126
- MRK-3m-loss: 1.747261002195295
- NVO-3m-loss: 2.041623352733811
- NVS-3m-loss: 1.2084892475952937
- PFE-3m-loss: 2.5518787621946375
- ROG-3m-loss: 13.757700842141588

### 2.3. 6-step-ahead VaR: In-sample forecast for 6 months

In [40]:
for t in tickers:
    loss = rolling_6m_forecast(data, dates6m, t, alpha=0.05, columns_to_exclude= to_exclude2)
    print(f'{t}-6m-loss: {loss} \n')

Iteration:      1,   Func. Count:      6,   Neg. LLF: 874.1785677557702
Iteration:      2,   Func. Count:     12,   Neg. LLF: 1200.4721531726964
Iteration:      3,   Func. Count:     19,   Neg. LLF: 870.6852289593519
Iteration:      4,   Func. Count:     26,   Neg. LLF: 867.8399619926613
Iteration:      5,   Func. Count:     32,   Neg. LLF: 867.828481407204
Iteration:      6,   Func. Count:     37,   Neg. LLF: 867.8058039900275
Iteration:      7,   Func. Count:     42,   Neg. LLF: 867.748173849943
Iteration:      8,   Func. Count:     47,   Neg. LLF: 867.7462745676646
Iteration:      9,   Func. Count:     52,   Neg. LLF: 867.7461706287772
Iteration:     10,   Func. Count:     57,   Neg. LLF: 867.746169625292
Iteration:     11,   Func. Count:     61,   Neg. LLF: 867.7461696251956
Optimization terminated successfully    (Exit mode 0)
            Current function value: 867.746169625292
            Iterations: 11
            Function evaluations: 61
            Gradient evaluations: 11
It

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      1,   Func. Count:      6,   Neg. LLF: 947.3225299769143
Iteration:      2,   Func. Count:     14,   Neg. LLF: 880.899936039578
Iteration:      3,   Func. Count:     20,   Neg. LLF: 878.1339497577962
Iteration:      4,   Func. Count:     27,   Neg. LLF: 876.6737053132299
Iteration:      5,   Func. Count:     32,   Neg. LLF: 876.6672585066991
Iteration:      6,   Func. Count:     37,   Neg. LLF: 876.6384410362984
Iteration:      7,   Func. Count:     42,   Neg. LLF: 876.589180523503
Iteration:      8,   Func. Count:     47,   Neg. LLF: 876.587517374675
Iteration:      9,   Func. Count:     52,   Neg. LLF: 876.5873820200592
Iteration:     10,   Func. Count:     57,   Neg. LLF: 876.5873810789516
Optimization terminated successfully    (Exit mode 0)
            Current function value: 876.5873810789516
            Iterations: 10
            Function evaluations: 57
            Gradient evaluations: 10
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1222.9902668269392
I

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      5,   Func. Count:     33,   Neg. LLF: 889.7675789803586
Iteration:      6,   Func. Count:     38,   Neg. LLF: 889.7537238994715
Iteration:      7,   Func. Count:     43,   Neg. LLF: 889.7361309667915
Iteration:      8,   Func. Count:     48,   Neg. LLF: 889.7274260931832
Iteration:      9,   Func. Count:     53,   Neg. LLF: 889.725416391847
Iteration:     10,   Func. Count:     58,   Neg. LLF: 889.7253145862583
Iteration:     11,   Func. Count:     62,   Neg. LLF: 889.7253145862426
Optimization terminated successfully    (Exit mode 0)
            Current function value: 889.7253145862583
            Iterations: 11
            Function evaluations: 62
            Gradient evaluations: 11
AZN-6m-loss: 4.903444131504825 

Iteration:      1,   Func. Count:      6,   Neg. LLF: 904.2346809763922
Iteration:      2,   Func. Count:     13,   Neg. LLF: 909.8940809007032
Iteration:      3,   Func. Count:     19,   Neg. LLF: 901.9676011216195
Iteration:      4,   Func. Count:     2

...

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1215.0219667989172
Iteration:      2,   Func. Count:     13,   Neg. LLF: 1014.7641046057527
Iteration:      3,   Func. Count:     19,   Neg. LLF: 1012.9168432390336
Iteration:      4,   Func. Count:     24,   Neg. LLF: 1012.916795479779
Iteration:      5,   Func. Count:     29,   Neg. LLF: 1012.9167683899902
Iteration:      6,   Func. Count:     34,   Neg. LLF: 1012.9167174245744
Iteration:      7,   Func. Count:     39,   Neg. LLF: 1012.9163417193431
Iteration:      8,   Func. Count:     44,   Neg. LLF: 1012.915136468405
Iteration:      9,   Func. Count:     49,   Neg. LLF: 1012.9149662892071
Iteration:     10,   Func. Count:     54,   Neg. LLF: 1012.914940677221
Iteration:     11,   Func. Count:     58,   Neg. LLF: 1012.9149406775548
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1012.914940677221
            Iterations: 11
            Function evaluations: 58
            Gradient evaluat

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      4,   Func. Count:     24,   Neg. LLF: 1023.1942387003072
Iteration:      5,   Func. Count:     29,   Neg. LLF: 1023.1941278027645
Iteration:      6,   Func. Count:     34,   Neg. LLF: 1023.1941125884837
Iteration:      7,   Func. Count:     39,   Neg. LLF: 1023.1940196342322
Iteration:      8,   Func. Count:     44,   Neg. LLF: 1023.1938083986237
Iteration:      9,   Func. Count:     49,   Neg. LLF: 1023.1934615938222
Iteration:     10,   Func. Count:     54,   Neg. LLF: 1023.1930778873815
Iteration:     11,   Func. Count:     59,   Neg. LLF: 1023.1928755673607
Iteration:     12,   Func. Count:     64,   Neg. LLF: 1023.1928461598402
Iteration:     13,   Func. Count:     69,   Neg. LLF: 1023.1928437372644
Iteration:     14,   Func. Count:     73,   Neg. LLF: 1023.192843737322
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1023.1928437372644
            Iterations: 14
            Function evaluations: 73
            Gradient eval

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Results for the in-sample forecast at 6 months of the naïve GARCH model only including the stock related variables:
- AZN-6m-loss: 4.903444131504825
- BMY-6m-loss: 3.145314758733963
- JNJ-6m-loss: 2.9673583449333343
- LLY-6m-loss: 3.955517166440173
- MRK-6m-loss: 3.8533075712325933
- NVO-6m-loss: 3.8323239117351013
- NVS-6m-loss: 2.337982438881043
- PFE-6m-loss: 3.47848250024052
- ROG-6m-loss: 35.04284232898197

### 2.4. 9-step-ahead VaR: In-sample forecast for 9 months

In [41]:
for t in tickers:
    loss = rolling_9m_forecast(data, dates9m, t, alpha=0.05, columns_to_exclude= to_exclude2)
    print(f'{t}-9m-loss: {loss} \n')

Iteration:      1,   Func. Count:      6,   Neg. LLF: 874.1785677557702
Iteration:      2,   Func. Count:     12,   Neg. LLF: 1200.4721531726964
Iteration:      3,   Func. Count:     19,   Neg. LLF: 870.6852289593519
Iteration:      4,   Func. Count:     26,   Neg. LLF: 867.8399619926613
Iteration:      5,   Func. Count:     32,   Neg. LLF: 867.828481407204
Iteration:      6,   Func. Count:     37,   Neg. LLF: 867.8058039900275
Iteration:      7,   Func. Count:     42,   Neg. LLF: 867.748173849943
Iteration:      8,   Func. Count:     47,   Neg. LLF: 867.7462745676646
Iteration:      9,   Func. Count:     52,   Neg. LLF: 867.7461706287772
Iteration:     10,   Func. Count:     57,   Neg. LLF: 867.746169625292
Iteration:     11,   Func. Count:     61,   Neg. LLF: 867.7461696251956
Optimization terminated successfully    (Exit mode 0)
            Current function value: 867.746169625292
            Iterations: 11
            Function evaluations: 61
            Gradient evaluations: 11
It

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter


Iteration:      7,   Func. Count:     42,   Neg. LLF: 873.2537478418432
Iteration:      8,   Func. Count:     47,   Neg. LLF: 873.2366633570832
Iteration:      9,   Func. Count:     52,   Neg. LLF: 873.2321873867108
Iteration:     10,   Func. Count:     57,   Neg. LLF: 873.2321447169634
Iteration:     11,   Func. Count:     61,   Neg. LLF: 873.2321447170441
Optimization terminated successfully    (Exit mode 0)
            Current function value: 873.2321447169634
            Iterations: 11
            Function evaluations: 61
            Gradient evaluations: 11
Iteration:      1,   Func. Count:      6,   Neg. LLF: 947.3225299769143
Iteration:      2,   Func. Count:     14,   Neg. LLF: 880.899936039578
Iteration:      3,   Func. Count:     20,   Neg. LLF: 878.1339497577962
Iteration:      4,   Func. Count:     27,   Neg. LLF: 876.6737053132299
Iteration:      5,   Func. Count:     32,   Neg. LLF: 876.6672585066991
Iteration:      6,   Func. Count:     37,   Neg. LLF: 876.6384410362984

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      2,   Func. Count:     13,   Neg. LLF: 913.222709191811
Iteration:      3,   Func. Count:     19,   Neg. LLF: 904.0742102389393
Iteration:      4,   Func. Count:     26,   Neg. LLF: 888.0176227402903
Iteration:      5,   Func. Count:     31,   Neg. LLF: 888.0170831290263
Iteration:      6,   Func. Count:     36,   Neg. LLF: 888.0170406502615
Iteration:      7,   Func. Count:     41,   Neg. LLF: 888.0167817047684
Iteration:      8,   Func. Count:     46,   Neg. LLF: 888.0154865930697
Iteration:      9,   Func. Count:     51,   Neg. LLF: 888.0109869234884
Iteration:     10,   Func. Count:     56,   Neg. LLF: 888.009594587629
Iteration:     11,   Func. Count:     61,   Neg. LLF: 888.0095017604818
Iteration:     12,   Func. Count:     66,   Neg. LLF: 888.0094967912892
Iteration:     13,   Func. Count:     70,   Neg. LLF: 888.0094967911114
Optimization terminated successfully    (Exit mode 0)
            Current function value: 888.0094967912892
            Iterations: 13
   

...

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter


Iteration:      9,   Func. Count:     54,   Neg. LLF: 799.6491473533056
Iteration:     10,   Func. Count:     58,   Neg. LLF: 799.6491473534852
Optimization terminated successfully    (Exit mode 0)
            Current function value: 799.6491473533056
            Iterations: 10
            Function evaluations: 58
            Gradient evaluations: 10
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1093.2920407111028
Iteration:      2,   Func. Count:     13,   Neg. LLF: 1182.2518284332864
Iteration:      3,   Func. Count:     21,   Neg. LLF: 943.1233514239641
Iteration:      4,   Func. Count:     28,   Neg. LLF: 803.2147680903146
Iteration:      5,   Func. Count:     33,   Neg. LLF: 803.303113863222
Iteration:      6,   Func. Count:     39,   Neg. LLF: 803.1659410111486
Iteration:      7,   Func. Count:     44,   Neg. LLF: 803.1636095468889
Iteration:      8,   Func. Count:     49,   Neg. LLF: 803.1635676718954
Iteration:      9,   Func. Count:     54,   Neg. LLF: 803.16356724608

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1215.0219667989172
Iteration:      2,   Func. Count:     13,   Neg. LLF: 1014.7641046057527
Iteration:      3,   Func. Count:     19,   Neg. LLF: 1012.9168432390336
Iteration:      4,   Func. Count:     24,   Neg. LLF: 1012.916795479779
Iteration:      5,   Func. Count:     29,   Neg. LLF: 1012.9167683899902
Iteration:      6,   Func. Count:     34,   Neg. LLF: 1012.9167174245744
Iteration:      7,   Func. Count:     39,   Neg. LLF: 1012.9163417193431
Iteration:      8,   Func. Count:     44,   Neg. LLF: 1012.915136468405
Iteration:      9,   Func. Count:     49,   Neg. LLF: 1012.9149662892071
Iteration:     10,   Func. Count:     54,   Neg. LLF: 1012.914940677221
Iteration:     11,   Func. Count:     58,   Neg. LLF: 1012.9149406775548
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1012.914940677221
            Iterations: 11
            Function evaluations: 58
            Gradient evaluat

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Results for the in-sample forecast at 9 months of the naïve GARCH model only including the stock related variables:
- AZN-9m-loss: 6.615671740041865
- BMY-9m-loss: 4.273425518571841
- JNJ-9m-loss: 4.609001006719325
- LLY-9m-loss: 5.27799903274221
- MRK-9m-loss: 3.594498717303232
- NVO-9m-loss: 3.1426340253078324
- NVS-9m-loss: 1.3872351248346997
- PFE-9m-loss: 3.4260101010202124
- ROG-9m-loss: 6.497664225334868

### 2.5. 12-step-ahead VaR: In-sample forecast for 12 months

In [42]:
for t in tickers:
    loss = rolling_12m_forecast(data, dates12m, t, alpha=0.05, columns_to_exclude= to_exclude2)
    print(f'{t}-12m-loss: {loss} \n')

Iteration:      1,   Func. Count:      6,   Neg. LLF: 874.1785677557702
Iteration:      2,   Func. Count:     12,   Neg. LLF: 1200.4721531726964
Iteration:      3,   Func. Count:     19,   Neg. LLF: 870.6852289593519
Iteration:      4,   Func. Count:     26,   Neg. LLF: 867.8399619926613
Iteration:      5,   Func. Count:     32,   Neg. LLF: 867.828481407204
Iteration:      6,   Func. Count:     37,   Neg. LLF: 867.8058039900275
Iteration:      7,   Func. Count:     42,   Neg. LLF: 867.748173849943
Iteration:      8,   Func. Count:     47,   Neg. LLF: 867.7462745676646
Iteration:      9,   Func. Count:     52,   Neg. LLF: 867.7461706287772
Iteration:     10,   Func. Count:     57,   Neg. LLF: 867.746169625292
Iteration:     11,   Func. Count:     61,   Neg. LLF: 867.7461696251956
Optimization terminated successfully    (Exit mode 0)
            Current function value: 867.746169625292
            Iterations: 11
            Function evaluations: 61
            Gradient evaluations: 11
AZ

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

JNJ-12m-loss: 3.661510230117694 

Iteration:      1,   Func. Count:      6,   Neg. LLF: 2254.6788963778395
Iteration:      2,   Func. Count:     13,   Neg. LLF: 929.0930031736946
Iteration:      3,   Func. Count:     20,   Neg. LLF: 878.0421093681352
Iteration:      4,   Func. Count:     26,   Neg. LLF: 870.6150307198097
Iteration:      5,   Func. Count:     32,   Neg. LLF: 866.4009282746734
Iteration:      6,   Func. Count:     38,   Neg. LLF: 864.3692435053997
Iteration:      7,   Func. Count:     44,   Neg. LLF: 864.2128747997451
Iteration:      8,   Func. Count:     49,   Neg. LLF: 864.2120701026428
Iteration:      9,   Func. Count:     54,   Neg. LLF: 864.2120402448049
Iteration:     10,   Func. Count:     59,   Neg. LLF: 864.2120198115566
Iteration:     11,   Func. Count:     63,   Neg. LLF: 864.2120198117773
Optimization terminated successfully    (Exit mode 0)
            Current function value: 864.2120198115566
            Iterations: 11
            Function evaluations: 63
 

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

NVO-12m-loss: 6.069257575371678 

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1080.4196934290235
Iteration:      2,   Func. Count:     13,   Neg. LLF: 1195.3172330054554
Iteration:      3,   Func. Count:     20,   Neg. LLF: 922.1082321132839
Iteration:      4,   Func. Count:     27,   Neg. LLF: 793.3136938066898
Iteration:      5,   Func. Count:     32,   Neg. LLF: 793.2926467980653
Iteration:      6,   Func. Count:     37,   Neg. LLF: 793.2779258205579
Iteration:      7,   Func. Count:     42,   Neg. LLF: 793.2760831388185
Iteration:      8,   Func. Count:     47,   Neg. LLF: 793.276019352837
Iteration:      9,   Func. Count:     52,   Neg. LLF: 793.275986413634
Iteration:     10,   Func. Count:     56,   Neg. LLF: 793.2759864139291
Optimization terminated successfully    (Exit mode 0)
            Current function value: 793.275986413634
            Iterations: 10
            Function evaluations: 56
            Gradient evaluations: 10
NVS-12m-loss: 0.0833853608451455 

Ite

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Results for the in-sample forecast at 12 months of the naïve GARCH model only including the stock related variables:
- AZN-12m-loss: 5.392792071787939
- BMY-12m-loss: 7.613825408180726
- JNJ-12m-loss: 3.661510230117694
- LLY-12m-loss: 8.914244012274354
- MRK-12m-loss: 5.2504360367705525
- NVO-12m-loss: 6.069257575371678
- NVS-12m-loss: 0.0833853608451455
- PFE-12m-loss: 7.323187380168994
- ROG-12m-loss: 9.489380154529998

# ABBV's separate treatment modelling:
Due to date mismatching with the other companies

In [27]:
merged_df = pd.read_csv('merged_df.csv')
merged_df

,Date,ABBV,unemployment_rate,CPI,treasury_yield,GDP_growth,SP500_return,AZN,BMY,JNJ,LLY,MRK,NVO,NVS,PFE,ROG,inflation_change,unemp_change,treasury_yield_change
0,2013-03-01,10.455062,7.5,232.282,1.984211,0.397212,3.337507,14.582205,11.414669,7.978126,4.851792,3.440210,-7.714279,9.067089,5.443922,-0.188640,-0.655,-0.2,0.069449
1,2013-04-01,12.922975,7.6,231.797,1.957500,0.085246,2.374537,3.881563,-3.568848,4.538222,-2.482836,7.359867,11.497419,3.537347,0.727651,-10.457790,-0.485,0.1,-0.026711
2,2013-05-01,-6.438455,7.5,231.893,1.759091,-0.190605,2.361004,-1.290451,16.816781,-1.231989,-4.008663,-0.638313,-8.792401,-2.711502,-6.329562,8.513136,0.096,-0.1,-0.198409
3,2013-06-01,-3.162299,7.5,232.445,1.928182,0.620501,-1.853781,-7.707318,-2.868894,2.762464,-6.788137,-0.535372,-3.805087,-1.463219,3.722003,2.269287,0.552,0.0,0.169091
4,2013-07-01,10.014486,7.3,232.900,2.300000,0.210359,5.724173,7.230424,-3.244595,8.898141,8.122938,4.655361,9.001745,1.272809,4.355584,17.540151,0.455,-0.2,0.371818
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,2023-01-01,-8.576205,3.4,300.536,3.616190,0.390254,6.776820,-3.584079,0.972908,-7.489384,-5.928822,-2.549213,2.541749,-0.110227,-13.817335,16.968326,1.546,-0.1,-0.274810
119,2023-02-01,5.152609,3.6,301.648,3.531500,-0.442183,-2.514271,-0.290649,-4.328217,-6.217115,-9.568502,-1.089288,1.592445,-7.172811,-7.286115,5.451681,1.112,0.2,-0.084690
120,2023-03-01,3.554254,3.5,301.808,3.746842,-0.442183,3.313488,8.035329,0.507544,1.862736,10.703390,0.141189,12.873250,9.367574,0.566924,11.025813,0.160,-0.1,0.215342
121,2023-04-01,-5.176628,3.4,302.918,3.663043,-0.442183,1.985238,5.489119,-3.664707,5.612908,15.269915,9.289214,5.836894,16.334413,-4.681371,-1.517467,1.110,-0.1,-0.083799


In [33]:
# Creating lagged variables
tickers = ['ABBV', 'AZN', 'BMY', 'JNJ', 'LLY', 'MRK', 'NVO', 'NVS', 'PFE','ROG']
covs = ['unemployment_rate', 'CPI', 'treasury_yield', 'GDP_growth', 'SP500_return', 'inflation_change', 'unemp_change', 'treasury_yield_change']
period_covs = [1, 2, 3]
period_returns = [1,2]

abbv_data, lagged_covs = create_lags_cov(merged_df, covs, period=period_covs)
abbv_data, lagged_tickers = create_lags_returns(merged_df, tickers, period= period_returns)

In [34]:
# Creating dummy variables
abbv_data['Quater_dummy'] = 0
abbv_data['Quater_rippel'] = 0
abbv_data['Date'] = pd.to_datetime(abbv_data['Date'])
for index, row in abbv_data.iterrows():
    if row['Date'].month in [3, 6, 9, 12]:
        abbv_data.loc[index, 'Quater_dummy'] = 1
    if row['Date'].month in [1, 4, 7, 10]:
        abbv_data.loc[index, 'Quater_dummy'] = 1

In [35]:
abbv_data

,Date,ABBV,unemployment_rate,CPI,treasury_yield,GDP_growth,SP500_return,AZN,BMY,JNJ,...,NVS_lag1,NVS_lag2,PFE_lag1,PFE_lag2,ROG_lag1,ROG_lag2,ABBV_lag1,ABBV_lag2,Quater_dummy,Quater_rippel
0,2014-01-01,-6.778990,6.6,235.288,2.901905,-0.623776,-3.001038,6.956380,-5.345197,-3.406445,...,1.592509,2.024510,-2.719366,3.388696,-2.085657,3.034781,8.998948,0.900706,1,0
1,2014-02-01,4.231658,6.7,235.547,2.858095,0.457816,4.551575,6.708648,7.604547,4.125658,...,-1.629759,1.592509,-0.750897,-2.719366,-1.300812,-2.085657,-6.778990,8.998948,0,0
2,2014-03-01,0.962478,6.7,236.028,2.709474,-0.024507,0.386504,-1.504253,-3.384807,7.400011,...,5.197926,-1.629759,5.624987,-0.750897,6.589779,-1.300812,4.231658,-6.778990,1,0
3,2014-04-01,1.322979,6.2,236.468,2.723333,0.757831,1.139553,21.840322,-3.580345,3.115110,...,5.656239,5.197926,0.865250,5.624987,-3.523955,6.589779,0.962478,4.231658,1,0
4,2014-05-01,5.253861,6.3,236.918,2.705238,0.379735,2.320664,-8.665410,-0.004722,0.167826,...,2.258307,5.656239,-2.615190,0.865250,-3.844918,-3.523955,1.322979,0.962478,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,2023-01-01,-8.576205,3.4,300.536,3.616190,0.390254,6.776820,-3.584079,0.972908,-7.489384,...,1.295221,10.390734,3.090302,7.690670,9.446071,-53.665066,0.266786,11.215906,1,0
109,2023-02-01,5.152609,3.6,301.648,3.531500,-0.442183,-2.514271,-0.290649,-4.328217,-6.217115,...,-0.110227,1.295221,-13.817335,3.090302,16.968326,9.446071,-8.576205,0.266786,0,0
110,2023-03-01,3.554254,3.5,301.808,3.746842,-0.442183,3.313488,8.035329,0.507544,1.862736,...,-7.172811,-0.110227,-7.286115,-13.817335,5.451681,16.968326,5.152609,-8.576205,1,0
111,2023-04-01,-5.176628,3.4,302.918,3.663043,-0.442183,1.985238,5.489119,-3.664707,5.612908,...,9.367574,-7.172811,0.566924,-7.286115,11.025813,5.451681,3.554254,5.152609,1,0


## 3. Macros Only

In [36]:
abbv_macro = abbv_data.copy()
to_exclude = []

for t in tickers:
    to_exclude.append(t)
    to_exclude.append(f'{t}_lag1')
    to_exclude.append(f'{t}_lag2')

for covariate in covs:
    to_exclude.append(covariate)

to_exclude.append('Date')

### 3.1. 1-step-ahead VaR: In-sample forecast for 1 month

In [37]:
for t in tickers:
    if t == 'ABBV':
        loss = rolling_1m_forecast(abbv_data, dates1m, t, alpha=0.05, columns_to_exclude= to_exclude)
        print(f'{t}: {loss}')

Iteration:      1,   Func. Count:      6,   Neg. LLF: 391.9332464341047
Iteration:      2,   Func. Count:     12,   Neg. LLF: 349.48703011216224
Iteration:      3,   Func. Count:     17,   Neg. LLF: 397.53709672504823
Iteration:      4,   Func. Count:     23,   Neg. LLF: 368.9198183541086
Iteration:      5,   Func. Count:     29,   Neg. LLF: 360.09864195660754
Iteration:      6,   Func. Count:     35,   Neg. LLF: 355.9282138879649
Iteration:      7,   Func. Count:     41,   Neg. LLF: 352.72426320186594
Iteration:      8,   Func. Count:     47,   Neg. LLF: 350.986679214979
Iteration:      9,   Func. Count:     53,   Neg. LLF: 350.0029984183652
Iteration:     10,   Func. Count:     59,   Neg. LLF: 349.4042217181557
Iteration:     11,   Func. Count:     65,   Neg. LLF: 349.0543802060845
Iteration:     12,   Func. Count:     71,   Neg. LLF: 348.89015479416605
Iteration:     13,   Func. Count:     76,   Neg. LLF: 348.87669596888634
Iteration:     14,   Func. Count:     81,   Neg. LLF: 348.8

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      1,   Func. Count:      6,   Neg. LLF: 404.2105411845844
Iteration:      2,   Func. Count:     12,   Neg. LLF: 359.3921587656769
Iteration:      3,   Func. Count:     17,   Neg. LLF: 407.045848629077
Iteration:      4,   Func. Count:     23,   Neg. LLF: 376.2103743862143
Iteration:      5,   Func. Count:     29,   Neg. LLF: 368.3814940742934
Iteration:      6,   Func. Count:     35,   Neg. LLF: 364.83898179706296
Iteration:      7,   Func. Count:     41,   Neg. LLF: 362.1260047813247
Iteration:      8,   Func. Count:     47,   Neg. LLF: 360.6706473618631
Iteration:      9,   Func. Count:     53,   Neg. LLF: 359.82987403472134
Iteration:     10,   Func. Count:     59,   Neg. LLF: 359.33066385504964
Iteration:     11,   Func. Count:     65,   Neg. LLF: 358.93745949011816
Iteration:     12,   Func. Count:     70,   Neg. LLF: 358.87566072916366
Iteration:     13,   Func. Count:     75,   Neg. LLF: 358.8721340581217
Iteration:     14,   Func. Count:     80,   Neg. LLF: 358.87

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      1,   Func. Count:      6,   Neg. LLF: 416.78722173928753
Iteration:      2,   Func. Count:     12,   Neg. LLF: 369.57966068510103
Iteration:      3,   Func. Count:     17,   Neg. LLF: 370.5550752766295
Iteration:      4,   Func. Count:     23,   Neg. LLF: 385.64088126918625
Iteration:      5,   Func. Count:     29,   Neg. LLF: 378.50537202485197
Iteration:      6,   Func. Count:     35,   Neg. LLF: 375.2919848874774
Iteration:      7,   Func. Count:     41,   Neg. LLF: 372.55266100795103
Iteration:      8,   Func. Count:     47,   Neg. LLF: 371.03620786268715
Iteration:      9,   Func. Count:     53,   Neg. LLF: 370.1557701185922
Iteration:     10,   Func. Count:     59,   Neg. LLF: 369.6442750389509
Iteration:     11,   Func. Count:     65,   Neg. LLF: 369.17905533757175
Iteration:     12,   Func. Count:     70,   Neg. LLF: 369.1169752068842
Iteration:     13,   Func. Count:     75,   Neg. LLF: 369.1097802637085
Iteration:     14,   Func. Count:     80,   Neg. LLF: 369

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      1,   Func. Count:      6,   Neg. LLF: 429.145022206387
Iteration:      2,   Func. Count:     12,   Neg. LLF: 379.4865568928191
Iteration:      3,   Func. Count:     17,   Neg. LLF: 379.5568272158402
Iteration:      4,   Func. Count:     23,   Neg. LLF: 381.0252976572026
Iteration:      5,   Func. Count:     29,   Neg. LLF: 382.87976359109103
Iteration:      6,   Func. Count:     35,   Neg. LLF: 381.0491556785438
Iteration:      7,   Func. Count:     41,   Neg. LLF: 380.0532278441943
Iteration:      8,   Func. Count:     47,   Neg. LLF: 379.5456033849743
Iteration:      9,   Func. Count:     53,   Neg. LLF: 379.176033567924
Iteration:     10,   Func. Count:     59,   Neg. LLF: 379.0776535523505
Iteration:     11,   Func. Count:     64,   Neg. LLF: 379.07589127527706
Iteration:     12,   Func. Count:     69,   Neg. LLF: 379.07531869507744
Iteration:     13,   Func. Count:     74,   Neg. LLF: 379.0752015801932
Iteration:     14,   Func. Count:     79,   Neg. LLF: 379.07519

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


### 3.2. 3-step-ahead VaR: In-sample forecast for 3 months

In [38]:
for t in tickers:
    if t == 'ABBV':
        loss = rolling_3m_forecast(abbv_macro, dates3m, t, alpha=0.05, columns_to_exclude= to_exclude)
        print(f'{t}: {loss}')

Iteration:      1,   Func. Count:      6,   Neg. LLF: 391.9332464341047
Iteration:      2,   Func. Count:     12,   Neg. LLF: 349.48703011216224
Iteration:      3,   Func. Count:     17,   Neg. LLF: 397.53709672504823
Iteration:      4,   Func. Count:     23,   Neg. LLF: 368.9198183541086
Iteration:      5,   Func. Count:     29,   Neg. LLF: 360.09864195660754
Iteration:      6,   Func. Count:     35,   Neg. LLF: 355.9282138879649
Iteration:      7,   Func. Count:     41,   Neg. LLF: 352.72426320186594
Iteration:      8,   Func. Count:     47,   Neg. LLF: 350.986679214979
Iteration:      9,   Func. Count:     53,   Neg. LLF: 350.0029984183652
Iteration:     10,   Func. Count:     59,   Neg. LLF: 349.4042217181557
Iteration:     11,   Func. Count:     65,   Neg. LLF: 349.0543802060845
Iteration:     12,   Func. Count:     71,   Neg. LLF: 348.89015479416605
Iteration:     13,   Func. Count:     76,   Neg. LLF: 348.87669596888634
Iteration:     14,   Func. Count:     81,   Neg. LLF: 348.8

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      1,   Func. Count:      6,   Neg. LLF: 404.2105411845844
Iteration:      2,   Func. Count:     12,   Neg. LLF: 359.3921587656769
Iteration:      3,   Func. Count:     17,   Neg. LLF: 407.045848629077
Iteration:      4,   Func. Count:     23,   Neg. LLF: 376.2103743862143
Iteration:      5,   Func. Count:     29,   Neg. LLF: 368.3814940742934
Iteration:      6,   Func. Count:     35,   Neg. LLF: 364.83898179706296
Iteration:      7,   Func. Count:     41,   Neg. LLF: 362.1260047813247
Iteration:      8,   Func. Count:     47,   Neg. LLF: 360.6706473618631
Iteration:      9,   Func. Count:     53,   Neg. LLF: 359.82987403472134
Iteration:     10,   Func. Count:     59,   Neg. LLF: 359.33066385504964
Iteration:     11,   Func. Count:     65,   Neg. LLF: 358.93745949011816
Iteration:     12,   Func. Count:     70,   Neg. LLF: 358.87566072916366
Iteration:     13,   Func. Count:     75,   Neg. LLF: 358.8721340581217
Iteration:     14,   Func. Count:     80,   Neg. LLF: 358.87

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      1,   Func. Count:      6,   Neg. LLF: 416.78722173928753
Iteration:      2,   Func. Count:     12,   Neg. LLF: 369.57966068510103
Iteration:      3,   Func. Count:     17,   Neg. LLF: 370.5550752766295
Iteration:      4,   Func. Count:     23,   Neg. LLF: 385.64088126918625
Iteration:      5,   Func. Count:     29,   Neg. LLF: 378.50537202485197
Iteration:      6,   Func. Count:     35,   Neg. LLF: 375.2919848874774
Iteration:      7,   Func. Count:     41,   Neg. LLF: 372.55266100795103
Iteration:      8,   Func. Count:     47,   Neg. LLF: 371.03620786268715
Iteration:      9,   Func. Count:     53,   Neg. LLF: 370.1557701185922
Iteration:     10,   Func. Count:     59,   Neg. LLF: 369.6442750389509
Iteration:     11,   Func. Count:     65,   Neg. LLF: 369.17905533757175
Iteration:     12,   Func. Count:     70,   Neg. LLF: 369.1169752068842
Iteration:     13,   Func. Count:     75,   Neg. LLF: 369.1097802637085
Iteration:     14,   Func. Count:     80,   Neg. LLF: 369

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:     15,   Func. Count:     85,   Neg. LLF: 375.941490063937
Iteration:     16,   Func. Count:     90,   Neg. LLF: 375.9414893672402
Optimization terminated successfully    (Exit mode 0)
            Current function value: 375.9414893672402
            Iterations: 16
            Function evaluations: 90
            Gradient evaluations: 16
Iteration:      1,   Func. Count:      6,   Neg. LLF: 429.145022206387
Iteration:      2,   Func. Count:     12,   Neg. LLF: 379.4865568928191
Iteration:      3,   Func. Count:     17,   Neg. LLF: 379.5568272158402
Iteration:      4,   Func. Count:     23,   Neg. LLF: 381.0252976572026
Iteration:      5,   Func. Count:     29,   Neg. LLF: 382.87976359109103
Iteration:      6,   Func. Count:     35,   Neg. LLF: 381.0491556785438
Iteration:      7,   Func. Count:     41,   Neg. LLF: 380.0532278441943
Iteration:      8,   Func. Count:     47,   Neg. LLF: 379.5456033849743
Iteration:      9,   Func. Count:     53,   Neg. LLF: 379.176033567924
I

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


### 3.3. 6-step-ahead VaR: In-sample forecast for 6 months

In [39]:
for t in tickers:
    if t == 'ABBV':
        loss = rolling_6m_forecast(abbv_macro, dates6m, t, alpha=0.05, columns_to_exclude= to_exclude)
        print(f'{t}: {loss}')

Iteration:      1,   Func. Count:      6,   Neg. LLF: 391.9332464341047
Iteration:      2,   Func. Count:     12,   Neg. LLF: 349.48703011216224
Iteration:      3,   Func. Count:     17,   Neg. LLF: 397.53709672504823
Iteration:      4,   Func. Count:     23,   Neg. LLF: 368.9198183541086
Iteration:      5,   Func. Count:     29,   Neg. LLF: 360.09864195660754
Iteration:      6,   Func. Count:     35,   Neg. LLF: 355.9282138879649
Iteration:      7,   Func. Count:     41,   Neg. LLF: 352.72426320186594
Iteration:      8,   Func. Count:     47,   Neg. LLF: 350.986679214979
Iteration:      9,   Func. Count:     53,   Neg. LLF: 350.0029984183652
Iteration:     10,   Func. Count:     59,   Neg. LLF: 349.4042217181557
Iteration:     11,   Func. Count:     65,   Neg. LLF: 349.0543802060845
Iteration:     12,   Func. Count:     71,   Neg. LLF: 348.89015479416605
Iteration:     13,   Func. Count:     76,   Neg. LLF: 348.87669596888634
Iteration:     14,   Func. Count:     81,   Neg. LLF: 348.8

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      1,   Func. Count:      6,   Neg. LLF: 404.2105411845844
Iteration:      2,   Func. Count:     12,   Neg. LLF: 359.3921587656769
Iteration:      3,   Func. Count:     17,   Neg. LLF: 407.045848629077
Iteration:      4,   Func. Count:     23,   Neg. LLF: 376.2103743862143
Iteration:      5,   Func. Count:     29,   Neg. LLF: 368.3814940742934
Iteration:      6,   Func. Count:     35,   Neg. LLF: 364.83898179706296
Iteration:      7,   Func. Count:     41,   Neg. LLF: 362.1260047813247
Iteration:      8,   Func. Count:     47,   Neg. LLF: 360.6706473618631
Iteration:      9,   Func. Count:     53,   Neg. LLF: 359.82987403472134
Iteration:     10,   Func. Count:     59,   Neg. LLF: 359.33066385504964
Iteration:     11,   Func. Count:     65,   Neg. LLF: 358.93745949011816
Iteration:     12,   Func. Count:     70,   Neg. LLF: 358.87566072916366
Iteration:     13,   Func. Count:     75,   Neg. LLF: 358.8721340581217
Iteration:     14,   Func. Count:     80,   Neg. LLF: 358.87

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:     11,   Func. Count:     65,   Neg. LLF: 365.42503724397824
Iteration:     12,   Func. Count:     70,   Neg. LLF: 365.4024447178369
Iteration:     13,   Func. Count:     75,   Neg. LLF: 365.40084125645006
Iteration:     14,   Func. Count:     80,   Neg. LLF: 365.40073223421143
Iteration:     15,   Func. Count:     85,   Neg. LLF: 365.4006886500943
Iteration:     16,   Func. Count:     89,   Neg. LLF: 365.40068864862303
Optimization terminated successfully    (Exit mode 0)
            Current function value: 365.4006886500943
            Iterations: 16
            Function evaluations: 89
            Gradient evaluations: 16
Iteration:      1,   Func. Count:      6,   Neg. LLF: 416.78722173928753
Iteration:      2,   Func. Count:     12,   Neg. LLF: 369.57966068510103
Iteration:      3,   Func. Count:     17,   Neg. LLF: 370.5550752766295
Iteration:      4,   Func. Count:     23,   Neg. LLF: 385.64088126918625
Iteration:      5,   Func. Count:     29,   Neg. LLF: 378.505372

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


### 3.4. 9-step-ahead VaR: In-sample forecast for 9 months

In [40]:
for t in tickers:
    if t == 'ABBV':
        loss = rolling_9m_forecast(abbv_macro, dates9m, t, alpha=0.05, columns_to_exclude= to_exclude)
        print(f'{t}: {loss}')

Iteration:      1,   Func. Count:      6,   Neg. LLF: 391.9332464341047
Iteration:      2,   Func. Count:     12,   Neg. LLF: 349.48703011216224
Iteration:      3,   Func. Count:     17,   Neg. LLF: 397.53709672504823
Iteration:      4,   Func. Count:     23,   Neg. LLF: 368.9198183541086
Iteration:      5,   Func. Count:     29,   Neg. LLF: 360.09864195660754
Iteration:      6,   Func. Count:     35,   Neg. LLF: 355.9282138879649
Iteration:      7,   Func. Count:     41,   Neg. LLF: 352.72426320186594
Iteration:      8,   Func. Count:     47,   Neg. LLF: 350.986679214979
Iteration:      9,   Func. Count:     53,   Neg. LLF: 350.0029984183652
Iteration:     10,   Func. Count:     59,   Neg. LLF: 349.4042217181557
Iteration:     11,   Func. Count:     65,   Neg. LLF: 349.0543802060845
Iteration:     12,   Func. Count:     71,   Neg. LLF: 348.89015479416605
Iteration:     13,   Func. Count:     76,   Neg. LLF: 348.87669596888634
Iteration:     14,   Func. Count:     81,   Neg. LLF: 348.8

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      1,   Func. Count:      6,   Neg. LLF: 404.2105411845844
Iteration:      2,   Func. Count:     12,   Neg. LLF: 359.3921587656769
Iteration:      3,   Func. Count:     17,   Neg. LLF: 407.045848629077
Iteration:      4,   Func. Count:     23,   Neg. LLF: 376.2103743862143
Iteration:      5,   Func. Count:     29,   Neg. LLF: 368.3814940742934
Iteration:      6,   Func. Count:     35,   Neg. LLF: 364.83898179706296
Iteration:      7,   Func. Count:     41,   Neg. LLF: 362.1260047813247
Iteration:      8,   Func. Count:     47,   Neg. LLF: 360.6706473618631
Iteration:      9,   Func. Count:     53,   Neg. LLF: 359.82987403472134
Iteration:     10,   Func. Count:     59,   Neg. LLF: 359.33066385504964
Iteration:     11,   Func. Count:     65,   Neg. LLF: 358.93745949011816
Iteration:     12,   Func. Count:     70,   Neg. LLF: 358.87566072916366
Iteration:     13,   Func. Count:     75,   Neg. LLF: 358.8721340581217
Iteration:     14,   Func. Count:     80,   Neg. LLF: 358.87

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


### 3.5. 12-step-ahead VaR: In-sample forecast for 12 months

In [41]:
for t in tickers:
    if t == 'ABBV':
        loss = rolling_12m_forecast(abbv_macro, dates12m, t, alpha=0.05, columns_to_exclude= to_exclude)
        print(f'{t}: {loss}')

Iteration:      1,   Func. Count:      6,   Neg. LLF: 391.9332464341047
Iteration:      2,   Func. Count:     12,   Neg. LLF: 349.48703011216224
Iteration:      3,   Func. Count:     17,   Neg. LLF: 397.53709672504823
Iteration:      4,   Func. Count:     23,   Neg. LLF: 368.9198183541086
Iteration:      5,   Func. Count:     29,   Neg. LLF: 360.09864195660754
Iteration:      6,   Func. Count:     35,   Neg. LLF: 355.9282138879649
Iteration:      7,   Func. Count:     41,   Neg. LLF: 352.72426320186594
Iteration:      8,   Func. Count:     47,   Neg. LLF: 350.986679214979
Iteration:      9,   Func. Count:     53,   Neg. LLF: 350.0029984183652
Iteration:     10,   Func. Count:     59,   Neg. LLF: 349.4042217181557
Iteration:     11,   Func. Count:     65,   Neg. LLF: 349.0543802060845
Iteration:     12,   Func. Count:     71,   Neg. LLF: 348.89015479416605
Iteration:     13,   Func. Count:     76,   Neg. LLF: 348.87669596888634
Iteration:     14,   Func. Count:     81,   Neg. LLF: 348.8

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


## Ticker Data

In [42]:
abbv_stocks = abbv_data.copy()
to_exclude2 = []

for covariate in covs:
    to_exclude2.append(covariate)
    to_exclude2.append(f'{covariate}_lag1')
    to_exclude2.append(f'{covariate}_lag2')
    to_exclude2.append(f'{covariate}_lag3')

for t in tickers:
    to_exclude2.append(t)

to_exclude2.append('Date')

### 4.1. 1-step-ahead VaR: In-sample forecast for 1 month

In [43]:
for t in tickers:
    if t == 'ABBV':
        loss = rolling_1m_forecast(abbv_data, dates1m, t, alpha=0.05, columns_to_exclude= to_exclude2)
        print(f'{t}: {loss}')

Iteration:      1,   Func. Count:      6,   Neg. LLF: 391.9332464341047
Iteration:      2,   Func. Count:     12,   Neg. LLF: 349.48703011216224
Iteration:      3,   Func. Count:     17,   Neg. LLF: 397.53709672504823
Iteration:      4,   Func. Count:     23,   Neg. LLF: 368.9198183541086
Iteration:      5,   Func. Count:     29,   Neg. LLF: 360.09864195660754
Iteration:      6,   Func. Count:     35,   Neg. LLF: 355.9282138879649
Iteration:      7,   Func. Count:     41,   Neg. LLF: 352.72426320186594
Iteration:      8,   Func. Count:     47,   Neg. LLF: 350.986679214979
Iteration:      9,   Func. Count:     53,   Neg. LLF: 350.0029984183652
Iteration:     10,   Func. Count:     59,   Neg. LLF: 349.4042217181557
Iteration:     11,   Func. Count:     65,   Neg. LLF: 349.0543802060845
Iteration:     12,   Func. Count:     71,   Neg. LLF: 348.89015479416605
Iteration:     13,   Func. Count:     76,   Neg. LLF: 348.87669596888634
Iteration:     14,   Func. Count:     81,   Neg. LLF: 348.8

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:     16,   Func. Count:     90,   Neg. LLF: 351.9890130229751
Iteration:     17,   Func. Count:     94,   Neg. LLF: 351.9890130208906
Optimization terminated successfully    (Exit mode 0)
            Current function value: 351.9890130229751
            Iterations: 17
            Function evaluations: 94
            Gradient evaluations: 17
Iteration:      1,   Func. Count:      6,   Neg. LLF: 400.0525041734909
Iteration:      2,   Func. Count:     12,   Neg. LLF: 356.0155144912933
Iteration:      3,   Func. Count:     17,   Neg. LLF: 403.62253294886114
Iteration:      4,   Func. Count:     23,   Neg. LLF: 373.0661873954555
Iteration:      5,   Func. Count:     29,   Neg. LLF: 365.154610984219
Iteration:      6,   Func. Count:     35,   Neg. LLF: 361.6308637321232
Iteration:      7,   Func. Count:     41,   Neg. LLF: 358.83272237125595
Iteration:      8,   Func. Count:     47,   Neg. LLF: 357.3338650572392
Iteration:      9,   Func. Count:     53,   Neg. LLF: 356.467510836486

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:     11,   Func. Count:     65,   Neg. LLF: 362.0515272722149
Iteration:     12,   Func. Count:     71,   Neg. LLF: 361.9552338874048
Iteration:     13,   Func. Count:     76,   Neg. LLF: 361.94695464079444
Iteration:     14,   Func. Count:     81,   Neg. LLF: 361.94642518324855
Iteration:     15,   Func. Count:     86,   Neg. LLF: 361.94627935860603
Iteration:     16,   Func. Count:     91,   Neg. LLF: 361.9462785467863
Optimization terminated successfully    (Exit mode 0)
            Current function value: 361.9462785467863
            Iterations: 16
            Function evaluations: 91
            Gradient evaluations: 16
Iteration:      1,   Func. Count:      6,   Neg. LLF: 412.30084645332295
Iteration:      2,   Func. Count:     12,   Neg. LLF: 365.85452515883827
Iteration:      3,   Func. Count:     17,   Neg. LLF: 366.5314290748993
Iteration:      4,   Func. Count:     23,   Neg. LLF: 380.6140276315244
Iteration:      5,   Func. Count:     29,   Neg. LLF: 373.57620004

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:     10,   Func. Count:     59,   Neg. LLF: 372.17092404117693
Iteration:     11,   Func. Count:     64,   Neg. LLF: 372.16463420449764
Iteration:     12,   Func. Count:     69,   Neg. LLF: 372.16414390764623
Iteration:     13,   Func. Count:     74,   Neg. LLF: 372.16382650234044
Iteration:     14,   Func. Count:     79,   Neg. LLF: 372.1638257163136
Optimization terminated successfully    (Exit mode 0)
            Current function value: 372.1638257163136
            Iterations: 14
            Function evaluations: 79
            Gradient evaluations: 14
Iteration:      1,   Func. Count:      6,   Neg. LLF: 425.2036596861111
Iteration:      2,   Func. Count:     12,   Neg. LLF: 376.39377326274035
Iteration:      3,   Func. Count:     17,   Neg. LLF: 377.15660785751743
Iteration:      4,   Func. Count:     23,   Neg. LLF: 391.8841176485095
Iteration:      5,   Func. Count:     29,   Neg. LLF: 384.70493062476356
Iteration:      6,   Func. Count:     35,   Neg. LLF: 381.284918

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


### 4.2. 3-step-ahead VaR: In-sample forecast for 3 months

In [44]:
for t in tickers:
    if t == 'ABBV':
        loss = rolling_3m_forecast(abbv_data, dates3m, t, alpha=0.05, columns_to_exclude= to_exclude2)
        print(f'{t}: {loss}')

Iteration:      1,   Func. Count:      6,   Neg. LLF: 391.9332464341047
Iteration:      2,   Func. Count:     12,   Neg. LLF: 349.48703011216224
Iteration:      3,   Func. Count:     17,   Neg. LLF: 397.53709672504823
Iteration:      4,   Func. Count:     23,   Neg. LLF: 368.9198183541086
Iteration:      5,   Func. Count:     29,   Neg. LLF: 360.09864195660754
Iteration:      6,   Func. Count:     35,   Neg. LLF: 355.9282138879649
Iteration:      7,   Func. Count:     41,   Neg. LLF: 352.72426320186594
Iteration:      8,   Func. Count:     47,   Neg. LLF: 350.986679214979
Iteration:      9,   Func. Count:     53,   Neg. LLF: 350.0029984183652
Iteration:     10,   Func. Count:     59,   Neg. LLF: 349.4042217181557
Iteration:     11,   Func. Count:     65,   Neg. LLF: 349.0543802060845
Iteration:     12,   Func. Count:     71,   Neg. LLF: 348.89015479416605
Iteration:     13,   Func. Count:     76,   Neg. LLF: 348.87669596888634
Iteration:     14,   Func. Count:     81,   Neg. LLF: 348.8

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter


Iteration:      6,   Func. Count:     35,   Neg. LLF: 364.83898179706296
Iteration:      7,   Func. Count:     41,   Neg. LLF: 362.1260047813247
Iteration:      8,   Func. Count:     47,   Neg. LLF: 360.6706473618631
Iteration:      9,   Func. Count:     53,   Neg. LLF: 359.82987403472134
Iteration:     10,   Func. Count:     59,   Neg. LLF: 359.33066385504964
Iteration:     11,   Func. Count:     65,   Neg. LLF: 358.93745949011816
Iteration:     12,   Func. Count:     70,   Neg. LLF: 358.87566072916366
Iteration:     13,   Func. Count:     75,   Neg. LLF: 358.8721340581217
Iteration:     14,   Func. Count:     80,   Neg. LLF: 358.87123193685073
Iteration:     15,   Func. Count:     85,   Neg. LLF: 358.87112570959624
Iteration:     16,   Func. Count:     90,   Neg. LLF: 358.87111663651865
Iteration:     17,   Func. Count:     95,   Neg. LLF: 358.87111573559207
Optimization terminated successfully    (Exit mode 0)
            Current function value: 358.87111573559207
            Itera

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      1,   Func. Count:      6,   Neg. LLF: 416.78722173928753
Iteration:      2,   Func. Count:     12,   Neg. LLF: 369.57966068510103
Iteration:      3,   Func. Count:     17,   Neg. LLF: 370.5550752766295
Iteration:      4,   Func. Count:     23,   Neg. LLF: 385.64088126918625
Iteration:      5,   Func. Count:     29,   Neg. LLF: 378.50537202485197
Iteration:      6,   Func. Count:     35,   Neg. LLF: 375.2919848874774
Iteration:      7,   Func. Count:     41,   Neg. LLF: 372.55266100795103
Iteration:      8,   Func. Count:     47,   Neg. LLF: 371.03620786268715
Iteration:      9,   Func. Count:     53,   Neg. LLF: 370.1557701185922
Iteration:     10,   Func. Count:     59,   Neg. LLF: 369.6442750389509
Iteration:     11,   Func. Count:     65,   Neg. LLF: 369.17905533757175
Iteration:     12,   Func. Count:     70,   Neg. LLF: 369.1169752068842
Iteration:     13,   Func. Count:     75,   Neg. LLF: 369.1097802637085
Iteration:     14,   Func. Count:     80,   Neg. LLF: 369

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

### 4.3. 6-step-ahead VaR: In-sample forecast for 6 months

In [45]:
for t in tickers:
    if t == 'ABBV':
        loss = rolling_6m_forecast(abbv_data, dates6m, t, alpha=0.05, columns_to_exclude= to_exclude2)
        print(f'{t}: {loss}')

Iteration:      1,   Func. Count:      6,   Neg. LLF: 391.9332464341047
Iteration:      2,   Func. Count:     12,   Neg. LLF: 349.48703011216224
Iteration:      3,   Func. Count:     17,   Neg. LLF: 397.53709672504823
Iteration:      4,   Func. Count:     23,   Neg. LLF: 368.9198183541086
Iteration:      5,   Func. Count:     29,   Neg. LLF: 360.09864195660754
Iteration:      6,   Func. Count:     35,   Neg. LLF: 355.9282138879649
Iteration:      7,   Func. Count:     41,   Neg. LLF: 352.72426320186594
Iteration:      8,   Func. Count:     47,   Neg. LLF: 350.986679214979
Iteration:      9,   Func. Count:     53,   Neg. LLF: 350.0029984183652
Iteration:     10,   Func. Count:     59,   Neg. LLF: 349.4042217181557
Iteration:     11,   Func. Count:     65,   Neg. LLF: 349.0543802060845
Iteration:     12,   Func. Count:     71,   Neg. LLF: 348.89015479416605
Iteration:     13,   Func. Count:     76,   Neg. LLF: 348.87669596888634
Iteration:     14,   Func. Count:     81,   Neg. LLF: 348.8

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(



Iteration:     10,   Func. Count:     59,   Neg. LLF: 352.44409822263395
Iteration:     11,   Func. Count:     65,   Neg. LLF: 352.0652620390938
Iteration:     12,   Func. Count:     70,   Neg. LLF: 351.9986449426672
Iteration:     13,   Func. Count:     75,   Neg. LLF: 351.9940162896886
Iteration:     14,   Func. Count:     80,   Neg. LLF: 351.98921865470265
Iteration:     15,   Func. Count:     85,   Neg. LLF: 351.9890296060895
Iteration:     16,   Func. Count:     90,   Neg. LLF: 351.9890130229751
Iteration:     17,   Func. Count:     94,   Neg. LLF: 351.9890130208906
Optimization terminated successfully    (Exit mode 0)
            Current function value: 351.9890130229751
            Iterations: 17
            Function evaluations: 94
            Gradient evaluations: 17
Iteration:      1,   Func. Count:      6,   Neg. LLF: 400.0525041734909
Iteration:      2,   Func. Count:     12,   Neg. LLF: 356.0155144912933
Iteration:      3,   Func. Count:     17,   Neg. LLF: 403.6225329488

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


Iteration:      3,   Func. Count:     17,   Neg. LLF: 363.8643313242647
Iteration:      4,   Func. Count:     23,   Neg. LLF: 378.66273377170904
Iteration:      5,   Func. Count:     29,   Neg. LLF: 371.8361842349881
Iteration:      6,   Func. Count:     35,   Neg. LLF: 368.5160281057364
Iteration:      7,   Func. Count:     41,   Neg. LLF: 365.7306126047908
Iteration:      8,   Func. Count:     47,   Neg. LLF: 364.0932735589592
Iteration:      9,   Func. Count:     53,   Neg. LLF: 363.1355299779363
Iteration:     10,   Func. Count:     59,   Neg. LLF: 362.57263590160966
Iteration:     11,   Func. Count:     65,   Neg. LLF: 362.0515272722149
Iteration:     12,   Func. Count:     71,   Neg. LLF: 361.9552338874048
Iteration:     13,   Func. Count:     76,   Neg. LLF: 361.94695464079444
Iteration:     14,   Func. Count:     81,   Neg. LLF: 361.94642518324855
Iteration:     15,   Func. Count:     86,   Neg. LLF: 361.94627935860603
Iteration:     16,   Func. Count:     91,   Neg. LLF: 361.9

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

### 4.4. 9-step-ahead VaR: In-sample forecast for 9 months

In [46]:
for t in tickers:
    if t == 'ABBV':
        loss = rolling_9m_forecast(abbv_data, dates9m, t, alpha=0.05, columns_to_exclude= to_exclude2)
        print(f'{t}: {loss}')

Iteration:      1,   Func. Count:      6,   Neg. LLF: 391.9332464341047
Iteration:      2,   Func. Count:     12,   Neg. LLF: 349.48703011216224
Iteration:      3,   Func. Count:     17,   Neg. LLF: 397.53709672504823
Iteration:      4,   Func. Count:     23,   Neg. LLF: 368.9198183541086
Iteration:      5,   Func. Count:     29,   Neg. LLF: 360.09864195660754
Iteration:      6,   Func. Count:     35,   Neg. LLF: 355.9282138879649
Iteration:      7,   Func. Count:     41,   Neg. LLF: 352.72426320186594
Iteration:      8,   Func. Count:     47,   Neg. LLF: 350.986679214979
Iteration:      9,   Func. Count:     53,   Neg. LLF: 350.0029984183652
Iteration:     10,   Func. Count:     59,   Neg. LLF: 349.4042217181557
Iteration:     11,   Func. Count:     65,   Neg. LLF: 349.0543802060845
Iteration:     12,   Func. Count:     71,   Neg. LLF: 348.89015479416605
Iteration:     13,   Func. Count:     76,   Neg. LLF: 348.87669596888634
Iteration:     14,   Func. Count:     81,   Neg. LLF: 348.8

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      2,   Func. Count:     12,   Neg. LLF: 359.3921587656769
Iteration:      3,   Func. Count:     17,   Neg. LLF: 407.045848629077
Iteration:      4,   Func. Count:     23,   Neg. LLF: 376.2103743862143
Iteration:      5,   Func. Count:     29,   Neg. LLF: 368.3814940742934
Iteration:      6,   Func. Count:     35,   Neg. LLF: 364.83898179706296
Iteration:      7,   Func. Count:     41,   Neg. LLF: 362.1260047813247
Iteration:      8,   Func. Count:     47,   Neg. LLF: 360.6706473618631
Iteration:      9,   Func. Count:     53,   Neg. LLF: 359.82987403472134
Iteration:     10,   Func. Count:     59,   Neg. LLF: 359.33066385504964
Iteration:     11,   Func. Count:     65,   Neg. LLF: 358.93745949011816
Iteration:     12,   Func. Count:     70,   Neg. LLF: 358.87566072916366
Iteration:     13,   Func. Count:     75,   Neg. LLF: 358.8721340581217
Iteration:     14,   Func. Count:     80,   Neg. LLF: 358.87123193685073
Iteration:     15,   Func. Count:     85,   Neg. LLF: 358.8

### 4.5. 12-step-ahead VaR: In-sample forecast for 12 months

In [47]:
for t in tickers:
    if t == 'ABBV':
        loss = rolling_12m_forecast(abbv_data, dates12m, t, alpha=0.05, columns_to_exclude= to_exclude2)
        print(f'{t}: {loss}')

Iteration:      1,   Func. Count:      6,   Neg. LLF: 391.9332464341047
Iteration:      2,   Func. Count:     12,   Neg. LLF: 349.48703011216224
Iteration:      3,   Func. Count:     17,   Neg. LLF: 397.53709672504823
Iteration:      4,   Func. Count:     23,   Neg. LLF: 368.9198183541086
Iteration:      5,   Func. Count:     29,   Neg. LLF: 360.09864195660754
Iteration:      6,   Func. Count:     35,   Neg. LLF: 355.9282138879649
Iteration:      7,   Func. Count:     41,   Neg. LLF: 352.72426320186594
Iteration:      8,   Func. Count:     47,   Neg. LLF: 350.986679214979
Iteration:      9,   Func. Count:     53,   Neg. LLF: 350.0029984183652
Iteration:     10,   Func. Count:     59,   Neg. LLF: 349.4042217181557
Iteration:     11,   Func. Count:     65,   Neg. LLF: 349.0543802060845
Iteration:     12,   Func. Count:     71,   Neg. LLF: 348.89015479416605
Iteration:     13,   Func. Count:     76,   Neg. LLF: 348.87669596888634
Iteration:     14,   Func. Count:     81,   Neg. LLF: 348.8

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
